# Business Need

The business needs to be able to scrape Google News API for articles related to the outbreak of various diseases, parse the article for the locations of the outbreak, and send the locations to another service for Geolocation tracking purposes.

### What is the problem you are attempting to solve? 
Copied from the proposal sent to me by company, R Zero.
"(Some intro stuff)...

We call Doppler "the Health Forecast" - it's an app on your phone that you open up to see any health risks for where you are today. We've done the conceptual work and are now ready to move into real development.

We want you to help us create a system that retrieves information from a variety of sources through an RSS feed and processes it so the information can be smoothly incorporated into our platform.

Specifically, we want an application that:
* Retrieves information from an RSS feed for
    * Locations (for example, an article that mentioned Zika in the US would mention Miami Beach, Florida - we need that)
        * And, ideally processes the location for a geo-coordinate (latitude/longitude)
    * Diseases/Conditions (such as Zika, the flu, etc)
    * Number of cases, if included
    
* Records the information into an easily accessible format (a CSV file would be great for now)
    * Include the processed information and the link to the source

* Has thorough code comments throughout, so developers that extend it in the future can understand the logic behind it."

### How is your solution valuable?
The software will be purchased by R Zero to be extended and licensed out to healthcare institutions.

### What is your data source and how will you access it?
Data wil be gathered by scraping Google News API to get URLs to articles that discuss the onset, diagnoses, spread, etc. of disease. The articles will be scraped for locations, disease and conditions, and number of cases.

### What techniques from the course do you anticipate using?
Scraping to get the data as described above.

Unsupervised Neural Networks for Natural Language Processing of the articles to extract the important information.

Supervised Neural Networks to create a regression model that predicts the likelihood of a disease spreading to an area depending on its distance from the current outbreak. 

### What do you anticipate will be the biggest challenge you'll face?
Getting reliable data from the scraping process.


### Note:
Before running this program, the machine on which this program is run will need to have Python 3.6 (I suggest downloading Anaconda) and run "pip install newsapi-python" from the command line interface.

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import json
import requests
from scrapy.linkextractors import LinkExtractor

#Build a crawler to crawl Google's top news articles and pull articles mentioning the given set of diseases
class GoogleSpider(scrapy.Spider):
    name = "GS"
    
    allowed_domains = ['newsapi.org']
    
    # The API Key used here belongs to Matthew Kennedy. The business can register for an api key for free 
    # using this link: https://developers.google.com/maps/documentation/javascript/get-api-key
    start_urls = ['https://newsapi.org/v2/everything?q=Zika&apiKey=2613ce5e838a464b814b7d5b4c2e6bf8'
                  # Additional urls can go here. Change 'q=Zika' to another disease and add it to the list
                  # Links need to be in '' and comma separated.
                 ]
      
    # Identifying the information we want from the query response and extracting it using xpath.
    def parse(self, response):
        data = json.loads(response.body_as_unicode())
        data2 = []
        for article in data['articles']:
            yield {
                'url' : article['url']
                , 'title': article['title']
                , 'description': article['description']
                , 'publish_date': article['publishedAt']
            }
                

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    # The FEED_URI is the name of the file that will be saved
    'FEED_URI': 'HealthAppScraper.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'ROBOTSTXT_OBEY': False,
    # This will need to be changed to the business' name
    'USER_AGENT': 'MatthewGoogleNewsCrawler (makennedy626@gmail.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # We use CLOSESPIDER_PAGECOUNT to limit our scraper to the first 10 links.    
    # 'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(GoogleSpider)
process.start()
print('First 100 links extracted!')

:0: UserWarning: You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


First 100 links extracted!


In [3]:
from pprint import pprint

with open('C:\\Users\\mkennedy\\HealthAppScraper.json') as data_file:    
    data = json.load(data_file)
pprint(data)

[{'description': 'Zika virüsü, sivrisinek ısırığıyla bulaşır fakat virüsün '
                 'insandan insana geçip geçmediği hakkında bir bilgi mevcut '
                 "değil. İnsanlarda ilk kez 1954 yılında Nijerya'da görülen "
                 'Zika virüsü, insandan önce 1947... Devamı için tıklayınız',
  'publish_date': '2018-01-09T15:53:21Z',
  'title': 'Zika virüsü nedir? (Zika virüsü belirtileri neler?)',
  'url': 'https://www.sabah.com.tr/saglik/2018/01/09/zika-virusu-nedir-zika-virusu-belirtileri-neler'},
 {'description': 'Zika virüsünün Türkiye’nin bazı bölgelerine yerleştiği iddia '
                 'edildi. Peki zika virüsünün bulaştığı nasıl anlaşılır, hangi '
                 'belirtiler görülür? Uzmanlar virüsün belirtilerini açıkladı. '
                 'Sivrisineklerden insanlara bulaşan zika virüsü nedir? Zika '
                 'virüsü insanlara …',
  'publish_date': '2018-01-11T14:47:24Z',
  'title': 'Zika virüsü nedir? Zika virüsü insanlara nasıl bulaşır? Hamile

In [4]:
print(str(data[1]['description']))

Zika virüsünün Türkiye’nin bazı bölgelerine yerleştiği iddia edildi. Peki zika virüsünün bulaştığı nasıl anlaşılır, hangi belirtiler görülür? Uzmanlar virüsün belirtilerini açıkladı. Sivrisineklerden insanlara bulaşan zika virüsü nedir? Zika virüsü insanlara …


In [12]:
import pandas as pd
import numpy as np
# The Json is a list of dictionary values. This code will store it to a DataFrame.
results = {'Description': [], 'Publish Date': [], 'URL': []}
columns = results.keys()
df_data = pd.DataFrame(data=results, columns=columns)

for entry in data:
    #print(data[entry]['description'])
    description = entry['description']
    date = entry['publish_date']
    url = entry['url']
    results = {'Description':[description], 'Publish Date':[date], 'URL': [url]}
    df_data = df_data.append(pd.DataFrame(data=results, columns=results.keys()), ignore_index = False)

In [13]:
df_data

,Description,Publish Date,URL
0,"Zika virüsü, sivrisinek ısırığıyla bulaşır fak...",2018-01-09T15:53:21Z,https://www.sabah.com.tr/saglik/2018/01/09/zik...
0,Zika virüsünün Türkiye’nin bazı bölgelerine ye...,2018-01-11T14:47:24Z,http://www.sozcu.com.tr/2018/saglik/zika-virus...
0,Dünya üzerinde yayılım göstererek hayati tehli...,2018-01-10T07:33:00Z,http://www.haber7.com/guncel/haber/2520534-zik...
0,Did you ever wonder what happened to Zika? You...,2018-01-08T04:01:00Z,https://www.lewrockwell.com/2018/01/dr-david-b...
0,Monkeys who catch Zika virus through bites fro...,2017-12-13T21:11:05Z,https://www.sciencedaily.com/releases/2017/12/...
0,( University of Wisconsin-Madison ) Monkeys wh...,2017-12-13T05:00:00Z,https://www.eurekalert.org/pub_releases/2017-1...
0,"Zika virüsü, neden olduğu rahatsızlıklar neden...",2018-01-11T09:13:48Z,http://www.hurriyet.com.tr/zika-virusu-nedir-n...
0,Reuters US study sheds light on how Zika cause...,2017-12-13T23:28:54Z,https://www.reuters.com/article/us-health-zika...
0,Patch.com Sexually Transmitted Zika Case Found...,2017-11-03T19:30:20Z,https://patch.com/florida/miami/sexually-trans...
0,Los Angeles Times Los Angeles: 1st sexually tr...,2018-01-05T10:00:57Z,http://outbreaknewstoday.com/los-angeles-1st-s...


In [ ]:
############################################################################
###### Now, create a crawler to crawl the information of each url ##########
############################################################################

# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


# Make a list for the urls to be stored into from the dataframe
url_list = []
for entry in data['URL']:
    url_list.append(entry)
    
print(url_list)


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = [
        url_list
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'firstpage.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

# Trying a different attempt

In [1]:
#Import the necessary modules
import datetime
from newsapi import NewsApiClient
import json
import requests

# Matthew Kennedy's API Key, business will need to register for their own api and place it below (registration is free)
newsapi = NewsApiClient(api_key='2613ce5e838a464b814b7d5b4c2e6bf8')

now = datetime.datetime.now()
date = now.day

all_articles = newsapi.get_everything(q=['Zika', 'H1N1']
                                     #, from_paramater = '2018-01-08' #this can be changed to any date
                                     , to = date
                                     , language = 'en'
                                     )

In [2]:
data = all_articles['articles']

import pandas as pd
import numpy as np
# The Json is a list of dictionary values. This code will store it to a DataFrame.
results = {'Description': [], 'Publish Date': [], 'URL': []}
columns = results.keys()
df_data = pd.DataFrame(data=results, columns=columns)

for entry in data:
    #print(data[entry]['description'])
    description = entry['description']
    date = entry['publishedAt']
    url = entry['url']
    results = {'Description':[description], 'Publish Date':[date], 'URL': [url]}
    df_data = df_data.append(pd.DataFrame(data=results, columns=results.keys()), ignore_index = False)
    
df_data.head()

,Description,Publish Date,URL
0,Did you ever wonder what happened to Zika? You...,2018-01-08T04:01:00Z,https://www.lewrockwell.com/2018/01/dr-david-b...
0,Monkeys who catch Zika virus through bites fro...,2017-12-13T21:11:05Z,https://www.sciencedaily.com/releases/2017/12/...
0,( University of Wisconsin-Madison ) Monkeys wh...,2017-12-13T05:00:00Z,https://www.eurekalert.org/pub_releases/2017-1...
0,Reuters US study sheds light on how Zika cause...,2017-12-13T23:28:54Z,https://www.reuters.com/article/us-health-zika...
0,FiercePharma Takeda's Zika vaccine candidate w...,2018-01-29T20:12:01Z,https://www.fiercepharma.com/vaccines/takeda-s...


In [3]:
############################################################################
###### Now, create a crawler to crawl the information of each url ##########
############################################################################

# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


# Make a list for the urls to be stored into from the dataframe
url_list = []
for entry in df_data['URL']:
    url_list.append(entry)
    
#print(url_list)


class ESSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "ESS"
    
    # URL(s) to start with.
    start_urls = url_list

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        # for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want.
            yield {
                'everything': response.xpath('//text()').extract()
                #'all': response.xpath('/descendant-or-self::node()').extract()
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                #'name': article.xpath('header/h2/a/@title').extract_first(),
                #'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                #'text': article.xpath('//*[@id="post-669657"]/div[3]').extract()
                #,'body': article.xpath('//*[@id="single-post"]').extract()
                #'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'firstpage4.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(ESSpider)
process.start()
print('Success!')

:0: UserWarning: You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


Success!


In [4]:
print(url_list)

['https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/', 'https://www.sciencedaily.com/releases/2017/12/171213161105.htm', 'https://www.eurekalert.org/pub_releases/2017-12/uow-mib121317.php', 'https://www.reuters.com/article/us-health-zika-paralysis/u-s-study-sheds-light-on-how-zika-causes-nerve-disorder-idUSKBN1E735E', 'https://www.fiercepharma.com/vaccines/takeda-s-zika-vaccine-gets-fda-fast-track-though-virus-no-longer-emergency', 'https://in.reuters.com/article/us-health-zika-paralysis/u-s-study-sheds-light-on-how-zika-causes-nerve-disorder-idINKBN1E735E', 'https://www.laboratoryequipment.com/news/2017/12/cdc-report-infants-affected-zika-not-reaching-developmental-milestones', 'https://medicalxpress.com/news/2017-11-zika-outbreak.html', 'http://outbreaknewstoday.com/los-angeles-1st-sexually-transmitted-zika-infection-reported-13985/', 'https://uk.reuters.com/article/us-health-zika-birthdefects/more-birth-defects-seen-in-u-s-areas-whe

In [5]:
print(df_data['Publish Date'])

0    2018-01-08T04:01:00Z
0    2017-12-13T21:11:05Z
0    2017-12-13T05:00:00Z
0    2017-12-13T23:28:54Z
0    2018-01-29T20:12:01Z
0    2017-12-13T23:33:03Z
0    2017-12-15T13:47:00Z
0    2017-11-17T11:45:03Z
0    2018-01-05T10:00:57Z
0    2018-01-25T18:27:31Z
0    2018-01-26T08:56:34Z
0    2018-01-26T11:14:53Z
0    2018-01-26T21:04:17Z
0    2017-12-07T03:18:00Z
0    2017-12-13T19:00:00Z
0    2017-12-13T19:00:44Z
0    2017-12-05T15:04:48Z
0    2017-12-15T17:22:14Z
0    2017-12-14T23:25:19Z
0    2017-12-14T18:17:00Z
Name: Publish Date, dtype: object


In [ ]:
html_doc = <body id="single-post" class="article-template-default single single-article postid-669657 single-author singular"><span id="fb-root"></span>
		<div class="wrapper">
			<div class="header-wrap clearfix">
				<div class="header inner clearfix">
                	<!-- A4 Cache --><!-- A4:A4:Non-Original --><div data-ad-loc="A4"><div id="div-gpt-ad-1509589843975-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1509589843975-0'); });
</script>
</div></div>                    <!-- mobile-ad Cache --><!-- MOBILE-AD:N/A:Non-Original --><div data-ad-loc="MOBILE-AD"><iframe style="border: none; display: none !important;" src="//rcm-na.amazon-adsystem.com/e/cm?t=lrc18-20&amp;o=1&amp;p=288&amp;l=ur1&amp;category=topratedproducts&amp;banner=14RX1TYMN6HK8B515XR2&amp;f=ifr&amp;linkID=4KOFNDEZHPLSNLXV" width="320" height="50" frameborder="0" marginwidth="0" scrolling="no" hidden="" kwframeid="1"></iframe></div>                    <div class="nav-wrap clearfix clear-both">
                        <div class="inner">
                            <div class="toggle-menu nav menu-main-menu-container"><a href="#">Show Menu</a></div>
                            <div class="main-menu-wrapper">
								<div id="main-menu-container" class="menu-main-menu-container"><ul id="menu-main-menu" class="nav"><li id="menu-item-441343" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441343"><a href="/archives/">Archives</a></li>
<li id="menu-item-441344" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441344"><a href="/columnists">Authors</a></li>
<li id="menu-item-441520" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441520"><a href="/lrc-blog/">Blog</a></li>
<li id="menu-item-441345" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-has-children menu-item-441345"><a href="#">Books &amp; Resources</a>
<ul class="sub-menu">
	<li id="menu-item-655521" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-655521"><a href="https://donate.lewrockwell.com/collections/books-by-lew-rockwell">Lew Rockwell Books</a></li>
	<li id="menu-item-441351" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441351"><a href="https://www.lewrockwell.com/books-resources/ron-paul-books/">Ron Paul Books</a></li>
	<li id="menu-item-441350" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441350"><a href="https://www.lewrockwell.com/books-resources/murray-n-rothbard-library-and-resources/">Murray N. Rothbard Library &amp; Resources</a></li>
	<li id="menu-item-441352" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441352"><a href="https://www.mises.org/">Mises Institute</a></li>
	<li id="menu-item-441349" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441349"><a href="https://www.lewrockwell.com/books-resources/lrc-best-selling-books/">LRC Bestselling Books</a></li>
</ul>
</li>
<li id="menu-item-548797" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-548797"><a href="/political-theatre/">Political Theatre</a></li>
<li id="menu-item-547557" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-547557"><a href="/podcast/">Podcasts</a></li>
<li id="menu-item-523610" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-523610"><a href="/store/">Store</a></li>
<li id="menu-item-47847" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-47847"><a href="https://www.lewrockwell.com/about/">About</a></li>
<li id="menu-item-521748" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-521748"><a rel="nofollow" href="/contact/">Contact</a></li>
<li id="menu-item-521749" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-521749"><a rel="nofollow" href="https://donate.lewrockwell.com/products/donations">Donate</a></li>
<li id="menu-item-523960" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-523960"><a href="https://www.lewrockwell.com/advertise-on-lrc/">Advertise</a></li>
<li class="menu-item menu-item-search"><form role="search" method="get" class="search-form" action="https://www.lewrockwell.com/"><label><span class="screen-reader-text">Search for:</span><input type="search" class="search-field" placeholder="Search …" value="" name="s" title="Search for:"></label><input type="submit" class="search-submit" value="Search"></form></li></ul></div>                            </div>
                                                    </div>
                    </div>
                	<div class="logo">
						<a href="https://www.lewrockwell.com/" title="LewRockwell" rel="home">LewRockwell<span>.com</span></a>
						<span>ANTI-STATE<em>•</em>ANTI-WAR<em>•</em>PRO-MARKET</span>
					</div>
                    	<div class="h-subscribe clearfix">
						<form method="post" action="//lewrockwell.us7.list-manage.com/subscribe/post?u=6ad24f4cd1574f1f7b8a0a03a&amp;id=d11da98377">
							<input type="email" name="EMAIL" id="Email" placeholder="Email address" class="placeholder">
							<input type="hidden" name="list" value="lewsdailypage">
							<input type="hidden" name="confirm" value="one">
							<input type="hidden" name="showconfirm" value="T">
							<input type="hidden" name="url" value="http://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/">
							<input type="submit" value="Subscribe" class="btn" name="subscribe">
						<span style="opacity: 1; left: 927.547px; top: 73.5px; width: 19px; min-width: 19px; height: 13px; position: absolute; background-image: url(&quot;data:image/svg+xml;base64,PHN2ZyB4bWxucz0naHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmcnIHhtbG5zOnhsaW5rPSdodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rJyB3aWR0aD0nMTcnIGhlaWdodD0nMTInIHZpZXdCb3g9JzAgMCAxNyAxMic+IDxkZWZzPiA8cGF0aCBpZD0nYScgZD0nTTcuOTA5IDEuNDYybDIuMTIxLjg2NHMtLjY3MS4xMy0xLjIwOS4yOTRjMCAwIC40MzcuNjM0Ljc3LjkzOC4zOTEtLjE4LjY1Ny0uMjQ4LjY1Ny0uMjQ4LS44MTEgMS42NjgtMi45NzkgMi43MDMtNC41MyAyLjcwMy0uMDkzIDAtLjQ4Mi0uMDA2LS43MjcuMDE1LS40MzUuMDIxLS41ODEuMzgtLjM3NC40NzMuMzczLjIwMSAxLjE0My42NjIuOTU4IDEuMDA5QzUuMiA4LjAwMy45OTkgMTEgLjk5OSAxMWwuNjQ4Ljg4Nkw2LjEyOSA4LjYzQzguNjAyIDYuOTQ4IDEyLjAwNiA2IDE1IDZoM1Y1aC00LjAwMWMtMS4wNTggMC0yLjA0LjEyMi0yLjQ3My0uMDItLjQwMi0uMTMzLS41MDItLjY3OS0uNDU1LTEuMDM1YTcuODcgNy44NyAwIDAgMSAuMTg3LS43MjljLjAyOC0uMDk5LjA0Ni0uMDc3LjE1NS0uMDk5LjU0LS4xMTIuNzc3LS4wOTUuODIxLS4xNi4xNDYtLjI0NS4yNTQtLjk3NC4yNTQtLjk3NEw3LjU2OS4zODlzLjIwMiAxLjAxMy4zNCAxLjA3M3onLz4gPC9kZWZzPiA8dXNlIGZpbGw9JyMwMDdDOTcnIGZpbGwtcnVsZT0nZXZlbm9kZCcgdHJhbnNmb3JtPSd0cmFuc2xhdGUoLTEpJyB4bGluazpocmVmPScjYScvPiA8L3N2Zz4=&quot;); background-repeat: no-repeat; background-position: 0px 0px; border: none; display: inline; visibility: visible; z-index: auto;"></span></form>
						<a href="https://twitter.com/#!/lewrockwell" target="_blank" class="ico-tw"></a>						
						<a href="https://www.facebook.com/LewRockwell" target="_blank" class="ico-fb"></a>
						<a href="https://www.lewrockwell.com//feed/" class="ico-rss"></a>
                        <!-- ad-header-right Cache --><!-- amz-small No Cache --><!-- AMZ-SMALL:N/A:Non-Original --><div data-ad-loc="AMZ-SMALL"><iframe style="border: none; display: none !important;" src="//rcm-na.amazon-adsystem.com/e/cm?t=lrc18-20&amp;o=1&amp;p=288&amp;l=ur1&amp;category=topratedproducts&amp;banner=14RX1TYMN6HK8B515XR2&amp;f=ifr&amp;linkID=4KOFNDEZHPLSNLXV" width="320" height="50" frameborder="0" marginwidth="0" scrolling="no" hidden="" kwframeid="2"></iframe></div>					</div>
				</div>
			</div>
			<div class="two-cols inner container clearfix">
			
                <div class="main-col">
        <div class="post-boxes-wrapper">				<div class="blog-box blog-post post-type-article" id="post-669657">
												<h2 class="blog-title"><a href="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/" rel="bookmark" title="Zika Schmika IV: Where Is Our Billion Dollars?">Zika Schmika IV: Where Is Our Billion Dollars?</a></h2>
												<div class="clearfix post-info">
														<p class="authors"> By <a href="https://www.lewrockwell.com/author/dr-david-brownstein/?ptype=article" title="Posts by Dr. David Brownstein" rel="author">Dr. David Brownstein</a></p>														<p class="attribution"><a href="http://www.drbrownstein.com">Dr. Brownstein's Holistic Medicine</a></p>							<p class="date">January 8, 2018</p>						</div>
						<div class="social-sharing">
			<span class="custom-buttons custom-buttons-merged">
				<a class="a2a_button_email" href="https://www.addtoany.com/add_to/email?linkurl=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F&amp;type=page&amp;linkname=Zika+Schmika+IV%3A+Where+Is+Our+Billion+Dollars%3F&amp;linknote=" title="Email" rel="nofollow" target="_blank"><!--img src="https://www.lewrockwell.com//wp-content/themes/lrc/images/icons/email.png" width="16" height="16" alt="Email"--></a>
				<a href="#" title="Print" rel="nofollow" target="_blank" class="print-page"><!--img src="https://www.lewrockwell.com//wp-content/themes/lrc/images/icons/print.png" width="16" height="16" alt="Print"--></a>
			</span><div class="a2a_kit a2a_kit_size_16 addtoany_list" data-a2a-url="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/" data-a2a-title="Zika Schmika" style="line-height: 16px;"><a class="a2a_button_facebook" href="/#facebook" title="Facebook" rel="nofollow noopener" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_facebook" style="background-color: rgb(59, 89, 152); width: 16px; line-height: 16px; height: 16px; background-size: 16px; border-radius: 2px;"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M17.78 27.5V17.008h3.522l.527-4.09h-4.05v-2.61c0-1.182.33-1.99 2.023-1.99h2.166V4.66c-.375-.05-1.66-.16-3.155-.16-3.123 0-5.26 1.905-5.26 5.405v3.016h-3.53v4.09h3.53V27.5h4.223z"></path></svg></span><span class="a2a_label">Facebook</span></a><a class="a2a_button_twitter" href="/#twitter" title="Twitter" rel="nofollow noopener" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_twitter" style="background-color: rgb(85, 172, 238); width: 16px; line-height: 16px; height: 16px; background-size: 16px; border-radius: 2px;"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M28 8.557a9.913 9.913 0 0 1-2.828.775 4.93 4.93 0 0 0 2.166-2.725 9.738 9.738 0 0 1-3.13 1.194 4.92 4.92 0 0 0-3.593-1.55 4.924 4.924 0 0 0-4.794 6.049c-4.09-.21-7.72-2.17-10.15-5.15a4.942 4.942 0 0 0-.665 2.477c0 1.71.87 3.214 2.19 4.1a4.968 4.968 0 0 1-2.23-.616v.06c0 2.39 1.7 4.38 3.952 4.83-.414.115-.85.174-1.297.174-.318 0-.626-.03-.928-.086a4.935 4.935 0 0 0 4.6 3.42 9.893 9.893 0 0 1-6.114 2.107c-.398 0-.79-.023-1.175-.068a13.953 13.953 0 0 0 7.55 2.213c9.056 0 14.01-7.507 14.01-14.013 0-.213-.005-.426-.015-.637.96-.695 1.795-1.56 2.455-2.55z"></path></svg></span><span class="a2a_label">Twitter</span></a><a class="a2a_button_twitter_tweet addtoany_special_service" data-url="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/" data-text="Zika Schmika"><iframe id="twitter-widget-0" scrolling="no" frameborder="0" allowtransparency="true" class="twitter-share-button twitter-share-button-rendered twitter-tweet-button" title="Twitter Tweet Button" src="https://platform.twitter.com/widgets/tweet_button.36c0c29c73929bf937f4c70adb1a29e4.en.html#dnt=false&amp;id=twitter-widget-0&amp;lang=en&amp;original_referer=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F&amp;related=AddToAny%2Cmicropat&amp;size=m&amp;text=Zika%20Schmika&amp;time=1518467185958&amp;type=share&amp;url=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F" style="position: static; visibility: visible; width: 60px; height: 20px;" kwframeid="6" data-url="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/"></iframe></a><a class="a2a_button_facebook_like addtoany_special_service" data-href="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/" style="width: 90px;"><div class="fb-like" data-href="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/" data-width="90" data-layout="button_count" data-ref="addtoany"></div></a><a class="a2a_button_google_plusone addtoany_special_service" data-annotation="none" data-href="https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/"><div id="___plusone_0" style="text-indent: 0px; margin: 0px; padding: 0px; background: transparent; border-style: none; float: none; line-height: normal; font-size: 1px; vertical-align: baseline; display: inline-block; width: 32px; height: 20px;"><iframe ng-non-bindable="" frameborder="0" hspace="0" marginheight="0" marginwidth="0" scrolling="no" style="position: static; top: 0px; width: 32px; margin: 0px; border-style: none; left: 0px; visibility: visible; height: 20px;" tabindex="0" vspace="0" width="100%" id="I0_1518467185593" name="I0_1518467185593" src="https://apis.google.com/u/0/se/0/_/+1/fastbutton?usegapi=1&amp;size=medium&amp;annotation=none&amp;hl=en-US&amp;origin=https%3A%2F%2Fwww.lewrockwell.com&amp;url=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F&amp;gsrc=3p&amp;ic=1&amp;jsh=m%3B%2F_%2Fscs%2Fapps-static%2F_%2Fjs%2Fk%3Doz.gapi.en.WfSuwKgCRTA.O%2Fm%3D__features__%2Fam%3DAQE%2Frt%3Dj%2Fd%3D1%2Frs%3DAGLTcCO0KuWtpejjm06jRxdbUYOYGDplMQ#_methods=onPlusOne%2C_ready%2C_close%2C_open%2C_resizeMe%2C_renderstart%2Concircled%2Cdrefresh%2Cerefresh%2Conload&amp;id=I0_1518467185593&amp;_gfid=I0_1518467185593&amp;parent=https%3A%2F%2Fwww.lewrockwell.com&amp;pfname=&amp;rpctoken=25474283" kwframeid="3" data-gapiattached="true" title="G+"></iframe></div></a><a class="a2a_dd addtoany_share_save addtoany_share" href="https://www.addtoany.com/share#url=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F&amp;title=Zika%20Schmika"><img src="https://static.addtoany.com/buttons/share_save_120_16.png" alt="Share"></a></div></div>						<div class="blog-content clearfix">
							                            	<!-- S3 Cache --><!-- S3:S3:Non-Original --><div data-ad-loc="S3"><div id="div-gpt-ad-1509590440695-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1509590440695-0'); });
</script>
</div></div>														<p></p><p>Did you ever wonder what happened to Zika?&nbsp; You should, as our Government gave the Centers for Disease Control and Prevention (CDC)&nbsp;<strong><u>$1.1 billion</u></strong>&nbsp;to study and develop a vaccine against Zika.&nbsp;&nbsp; Where did that money go and what happened to Zika?</p>
<p>Recall that the Zika virus was making national headlines in 2016 as the CDC and other Powers-That-Be were claiming that Zika was responsible for the microcephaly epidemic that was occurring in one area of Brazil.&nbsp; Here is a sample of what the media had to say about Zika in 2016:</p>
<p>I know many of you do not like the term ‘fake news’. (Trust me, I hear about it every time I use it, especially by my much older sister.)&nbsp; But, I can find no better way to describe the mainstream reporting &nbsp;Zika coverage by the media, which was being driven by the CDC.</p>
<p>Here’s what I wrote on&nbsp;<a href="http://blog.drbrownstein.com/zika-schmika-how-to-waste-2000000000/">July 8, 2016</a>:</p>
<p>“Zika refers to the virus spread by mosquitos.&nbsp; It was first described as coming from the Zika Forest of Uganda in 1947.&nbsp; Zika has affected many tropical countries including many in Africa, the Americas, Asia, and the Pacific.&nbsp; And, over the last 70 years, there has not been an epidemic of microcephaly—children born with small heads–in any of these countries until a few months ago.&nbsp; Microcephaly made headlines early this year when a Brazilian doctor reported an epidemic increase in children being born with small heads—microcephaly.&nbsp; This doctor also noticed that many of the children who were born with small heads had mothers who were infected with Zika during their pregnancy.”</p>
<p>I also discussed the fact that Zika occurred in many other countries which did not suffer with an outbreak of microcephaly.&nbsp; I suggested that Congress not fund the CDCs folly of studying a benign virus.</p>
<p>One month later I wrote a second&nbsp;<a href="http://blog.drbrownstein.com/zika-schmika-part-ii-the-story-gets-worse/">blog post</a>&nbsp;about Zika and commented that “…seven weeks after the CDC requested $1.9 billion dollars, the CDC announced that the Zika virus was, in fact, “…a cause of microcephaly and other severe fetal brain defects.”&nbsp; This ludicrous statement came even after the World Health Organization stated, “No scientific evidence to date confirms a link between Zika virus and microcephaly…”</p>
<p>A few weeks later, I&nbsp;<a href="http://blog.drbrownstein.com/zika-schmika-part-3-ridiculous-zika-vaccine-coming-soon/">posted</a>&nbsp;again about Zika lamenting that we (the taxpayers) were now funding Big Pharma to manufacture a vaccine against Zika even though:</p>
<ul>
<li>A microcephaly epidemic only occurred in NE Brazil</li>
<li>Zika was never shown to cause the microcephaly epidemic in NE Brazil</li>
<li>Zika is rampant throughout Brazil including the Amazon where there is no microcephaly epidemic</li>
<li>Colombia which is next to Brazil studied 12,000 newborn babies and found no microcephaly cases even though Zika is rampant there</li>
<li>A microcephaly epidemic occurred in NE Brazil, the CDC wants $1.9 billion to study and develop a vaccine from a virus that has NEVER been shown to cause microcephaly</li>
</ul>
<p>I finished that third post by stating, “I am hoping this is my last post about Zika.”</p>
<p>What prompted this fourth post?&nbsp; Two things.&nbsp; First, a patient asked me if she should be concerned about going to South America because of Zika.&nbsp; The answer to that one is easy, no concerns at all.</p>
<p style="text-align: center;"><a href="http://blog.drbrownstein.com/zika-schmika-iv-where-is-our-billion-dollars/"><span style="font-size: 14pt;"><strong>Read the Whole Article</strong></span></a></p>
<p></p>
                            								                                <p class="article-author-archives-link">
                                	<a href="https://www.lewrockwell.com/author/dr-david-brownstein/">The Best of Dr. David Brownstein</a>
                                </p>
							                                						</div>
						<div class="clearfix blog-footer blog-data">
							<p class="blog-tags"><span>Tags:</span><span class="blog-tags-links"></span></p>
                            							                        	    <div class="post-bio">David Brownstein, M.D. is a Board-Certified family physician and is one of the foremost practitioners of holistic medicine. He is the Medical Director of the Center for Holistic Medicine in West Bloomfield, MI. Dr. Brownstein has lectured internationally to physicians and others about his success in using natural hormones and nutritional therapies in his practice.</div>
                                <p class="post-copyright">Copyright © 2018 Dr. David Brownstein</p>
                                <p class="post-another-post">Previous article by Dr. David Brownstein:  <a href="https://www.lewrockwell.com/2017/11/dr-david-brownstein/new-hypertension-guidelines-same-old-song/" rel="next" class="next-post">New Hypertension Guidelines</a></p>                                                        						</div>
					</div>
					<div class="blog-footer clearfix"><a href="https://www.lewrockwell.com/2018/01/mark-sircus/begging-for-global-warming-as-global-cooling-becomes-obvious/" rel="prev" class="prev-post page-link prev-link">Begging for Global Warming</a><p></p><a href="https://www.lewrockwell.com/2018/01/no_author/how-to-stop-your-snoring/" rel="next" class="next-post page-link next-link">How To Stop Your Snoring</a><p></p></div></div>            <!-- L1 Cache --><!-- L1:L1:Non-Original --><div data-ad-loc="L1"><div id="div-gpt-ad-1509589986878-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1509589986878-0'); });
</script>
</div></div>    </div>
    <div class="right-col">
<!-- sidebar-right-non-original Cache --><h4><a href="https://www.lewrockwell.com/political-theatre/">Political Theatre</a></h4><ul class="list-articles"><li><a href="https://www.lewrockwell.com/political-theatre/peace/">Peace</a></li><li><a href="https://www.lewrockwell.com/political-theatre/revolutionary-beehive-crowdfunded/">Revolutionary Beehive Crowdfunded</a></li><li><a href="https://www.lewrockwell.com/political-theatre/hell-breaks-loose-california-high-school/">All Hell Breaks Loose at California High School</a></li><li><a href="https://www.lewrockwell.com/political-theatre/biased-fbi-doj-officials-broke-law/">Biased FBI and DOJ Officials Broke The Law</a></li><li><a href="https://www.lewrockwell.com/political-theatre/president-held-hostage/">A President Held Hostage</a></li><li><a href="https://www.lewrockwell.com/political-theatre/why-are-european-governments/">Why Are European Governments</a></li><li><a href="https://www.lewrockwell.com/political-theatre/dollar-treasurys-slide/">Dollar, Treasurys Slide</a></li><li><a href="https://www.lewrockwell.com/political-theatre/rising-debt-rising-rates/">Rising Debt + Rising Rates</a></li><li><a href="https://www.lewrockwell.com/political-theatre/germany-fine-social-media-firms/">Germany To Fine Social Media Firms</a></li><li><a href="https://www.lewrockwell.com/political-theatre/meet-us-armys-latest/">Meet The US Army’s Latest</a></li></ul><div class="textwidget custom-html-widget"><!-- burts-gold Cache --><!-- BURTS-GOLD:N/A:Non-Original --><div data-ad-loc="BURTS-GOLD"><a href="http://archive.lewrockwell.com/blumert/burt-gold.html"><img src="https://www.lewrockwell.com/wp-content/themes/lrc/images/burtspage300.png"></a></div></div><h4><a href="https://www.lewrockwell.com/lrc-blog/">LRC Blog</a></h4><ul class="list-articles"><li><a href="https://www.lewrockwell.com/lrc-blog/who-is-the-aggressor-in-syria/">Who Is The Aggressor In Syria?</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/americas-deified-politician/">America’s Most Deified Politician . . .</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/u-s-government-murdered-100-syrians-week/">The U.S. Government Murdered Over 100 Syrians This Week</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/select-committee-2016-2017-presidential-election-aftermath/">Select Committee on the 2016-2017 Presidential Election and its Aftermath</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/the-evils-of-slavery/">The Evils of Slavery</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/world-saved-sin/">How the World was Saved from Sin</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/remembering-burt-blumert-4/">Remembering Burt Blumert</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/libertarian-book-compilations/">Libertarian Book Compilations</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/they-were-all-out-to-get-trump/">They Were All Out to Get Trump</a></li><li><a href="https://www.lewrockwell.com/lrc-blog/u-s-israel-raising-war-risks-attacking-syria/">U.S. and Israel Raising War Risks by Attacking Syria</a></li></ul><h4><a href="https://www.lewrockwell.com/podcast/">LRC Podcasts</a></h4><ul class="list-articles"></ul><!-- S1 Cache --><!-- S1:S1:Non-Original --><div data-ad-loc="S1"><div id="div-gpt-ad-1509590146902-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1509590146902-0'); });
</script>
</div></div><!-- S2 Cache --><!-- S2:S2:Non-Original --><div data-ad-loc="S2"><div id="div-gpt-ad-1509590308757-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1509590308757-0'); });
</script>
</div></div></div>            </div>
		</div><div style="position: static;"><div class="a2a_overlay" id="a2apage_overlay"></div><div id="a2apage_modal" class="a2a_menu a2a_modal" role="dialog" tabindex="-1" aria-label="Copy link" style="display:none"><div class="a2a_copy_link_container"><span id="a2a_copy_link_icon" class="a2a_svg a2a_s_link" style="background-color:#0166ff" onclick="a2a.gEl('a2a_copy_link_text').click()"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M24.4 21.18c0-.36-.1-.67-.36-.92l-2.8-2.8a1.24 1.24 0 0 0-.92-.38c-.38 0-.7.14-.97.43.02.04.1.12.25.26l.3.3.2.24c.08.12.14.24.17.35.03.1.05.23.05.37 0 .36-.13.66-.38.92a1.25 1.25 0 0 1-.92.37 1.4 1.4 0 0 1-.37-.03 1.06 1.06 0 0 1-.35-.18 2.27 2.27 0 0 1-.25-.2 6.82 6.82 0 0 1-.3-.3l-.24-.25c-.3.28-.44.6-.44.98 0 .36.13.66.38.92l2.78 2.8c.24.23.54.35.9.35.37 0 .68-.12.93-.35l1.98-1.97c.26-.25.38-.55.38-.9zm-9.46-9.5c0-.37-.13-.67-.38-.92l-2.78-2.8a1.24 1.24 0 0 0-.9-.37c-.36 0-.67.1-.93.35L7.97 9.92c-.26.25-.38.55-.38.9 0 .36.1.67.37.92l2.8 2.8c.24.25.55.37.92.37.36 0 .7-.13.96-.4-.03-.04-.1-.12-.26-.26s-.24-.23-.3-.3a2.67 2.67 0 0 1-.2-.24 1.05 1.05 0 0 1-.17-.35 1.4 1.4 0 0 1-.04-.37c0-.36.1-.66.36-.9.26-.26.56-.4.92-.4.14 0 .26.03.37.06.12.03.23.1.35.17.1.1.2.16.25.2l.3.3.24.26c.3-.28.44-.6.44-.98zM27 21.17c0 1.07-.38 2-1.15 2.73l-1.98 1.98c-.74.75-1.66 1.12-2.73 1.12-1.1 0-2-.38-2.75-1.14l-2.8-2.8c-.74-.74-1.1-1.65-1.1-2.73 0-1.1.38-2.04 1.17-2.82l-1.18-1.17c-.8.8-1.72 1.18-2.82 1.18-1.08 0-2-.36-2.75-1.12l-2.8-2.8C5.38 12.8 5 11.9 5 10.82c0-1.08.38-2 1.15-2.74L8.13 6.1C8.87 5.37 9.78 5 10.86 5c1.1 0 2 .38 2.75 1.15l2.8 2.8c.74.73 1.1 1.65 1.1 2.72 0 1.1-.38 2.05-1.17 2.82l1.18 1.18c.8-.8 1.72-1.2 2.82-1.2 1.08 0 2 .4 2.75 1.14l2.8 2.8c.76.76 1.13 1.68 1.13 2.76z"></path></svg></span><input id="a2a_copy_link_text" type="text" title="Copy link"><div id="a2a_copy_link_copied">✓</div></div></div><div class="a2a_menu a2a_full" id="a2apage_full" role="dialog" tabindex="-1" aria-label="Share"><div class="a2a_full_header"><div id="a2apage_find_container" class="a2a_menu_find_container"><input id="a2apage_find" class="a2a_menu_find" type="text" onclick="a2a.focus_find()" onkeyup="a2a.do_find()" autocomplete="off" title="Find any service"><span id="a2apage_find_icon" class="a2a_svg a2a_s_find" onclick="a2a.focus_find()"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path fill="#CCC" d="M19.7 18.2l-4.5-4.5c.7-1.1 1.2-2.3 1.2-3.6 0-3.5-2.8-6.3-6.3-6.3s-6.3 2.8-6.3 6.3 2.8 6.3 6.3 6.3c1.4 0 2.6-.4 3.6-1.2l4.5 4.5c.6.6 1.3.7 1.7.2.5-.4.4-1.1-.2-1.7zm-9.6-3.6c-2.5 0-4.5-2.1-4.5-4.5 0-2.5 2.1-4.5 4.5-4.5 2.5 0 4.5 2.1 4.5 4.5s-2 4.5-4.5 4.5z"></path></svg></span></div></div><div class="a2a_full_services" id="a2apage_full_services" role="presentation"><a rel="nofollow noopener" class="a2a_i" href="/#facebook" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_facebook" style="background-color: rgb(59, 89, 152);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M17.78 27.5V17.008h3.522l.527-4.09h-4.05v-2.61c0-1.182.33-1.99 2.023-1.99h2.166V4.66c-.375-.05-1.66-.16-3.155-.16-3.123 0-5.26 1.905-5.26 5.405v3.016h-3.53v4.09h3.53V27.5h4.223z"></path></svg></span>Facebook </a><a rel="nofollow noopener" class="a2a_i" href="/#twitter" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_twitter" style="background-color: rgb(85, 172, 238);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M28 8.557a9.913 9.913 0 0 1-2.828.775 4.93 4.93 0 0 0 2.166-2.725 9.738 9.738 0 0 1-3.13 1.194 4.92 4.92 0 0 0-3.593-1.55 4.924 4.924 0 0 0-4.794 6.049c-4.09-.21-7.72-2.17-10.15-5.15a4.942 4.942 0 0 0-.665 2.477c0 1.71.87 3.214 2.19 4.1a4.968 4.968 0 0 1-2.23-.616v.06c0 2.39 1.7 4.38 3.952 4.83-.414.115-.85.174-1.297.174-.318 0-.626-.03-.928-.086a4.935 4.935 0 0 0 4.6 3.42 9.893 9.893 0 0 1-6.114 2.107c-.398 0-.79-.023-1.175-.068a13.953 13.953 0 0 0 7.55 2.213c9.056 0 14.01-7.507 14.01-14.013 0-.213-.005-.426-.015-.637.96-.695 1.795-1.56 2.455-2.55z"></path></svg></span>Twitter </a><a rel="nofollow noopener" class="a2a_i" href="/#google_plus" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_google_plus" style="background-color: rgb(221, 75, 57);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M27 15h-2v-2h-2v2h-2v2h2v2h2v-2h2m-15-2v2.4h3.97c-.16 1.03-1.2 3.02-3.97 3.02-2.39 0-4.34-1.98-4.34-4.42s1.95-4.42 4.34-4.42c1.36 0 2.27.58 2.79 1.08l1.9-1.83C15.47 9.69 13.89 9 12 9c-3.87 0-7 3.13-7 7s3.13 7 7 7c4.04 0 6.72-2.84 6.72-6.84 0-.46-.05-.81-.11-1.16H12z" fill="#FFF"></path></svg></span>Google+ </a><a rel="nofollow noopener" class="a2a_i" href="/#pinterest" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_pinterest" style="background-color: rgb(189, 8, 28);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.539 4.5c-6.277 0-9.442 4.5-9.442 8.253 0 2.272.86 4.293 2.705 5.046.303.125.574.005.662-.33.061-.231.205-.816.27-1.06.088-.331.053-.447-.191-.736-.532-.627-.873-1.439-.873-2.591 0-3.338 2.498-6.327 6.505-6.327 3.548 0 5.497 2.168 5.497 5.062 0 3.81-1.686 7.025-4.188 7.025-1.382 0-2.416-1.142-2.085-2.545.397-1.674 1.166-3.48 1.166-4.689 0-1.081-.581-1.983-1.782-1.983-1.413 0-2.548 1.462-2.548 3.419 0 1.247.421 2.091.421 2.091l-1.699 7.199c-.505 2.137-.076 4.755-.039 5.019.021.158.223.196.314.077.13-.17 1.813-2.247 2.384-4.324.162-.587.929-3.631.929-3.631.46.876 1.801 1.646 3.227 1.646 4.247 0 7.128-3.871 7.128-9.053.003-3.918-3.317-7.568-8.361-7.568z"></path></svg></span>Pinterest </a><a rel="nofollow noopener" class="a2a_i" href="/#email" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_email" style="background-color: rgb(1, 102, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M26 21.25v-9s-9.1 6.35-9.984 6.68C15.144 18.616 6 12.25 6 12.25v9c0 1.25.266 1.5 1.5 1.5h17c1.266 0 1.5-.22 1.5-1.5zm-.015-10.765c0-.91-.265-1.235-1.485-1.235h-17c-1.255 0-1.5.39-1.5 1.3l.015.14s9.035 6.22 10 6.56c1.02-.395 9.985-6.7 9.985-6.7l-.015-.065z"></path></svg></span>Email </a><a rel="nofollow noopener" class="a2a_i" href="/#linkedin" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_linkedin" style="background-color: rgb(0, 123, 181);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M6.227 12.61h4.19v13.48h-4.19V12.61zm2.095-6.7a2.43 2.43 0 0 1 0 4.86c-1.344 0-2.428-1.09-2.428-2.43s1.084-2.43 2.428-2.43m4.72 6.7h4.02v1.84h.058c.56-1.058 1.927-2.176 3.965-2.176 4.238 0 5.02 2.792 5.02 6.42v7.395h-4.183v-6.56c0-1.564-.03-3.574-2.178-3.574-2.18 0-2.514 1.7-2.514 3.46v6.668h-4.187V12.61z" fill="#FFF"></path></svg></span>LinkedIn </a><a rel="nofollow noopener" class="a2a_i" href="/#reddit" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_reddit" style="background-color: rgb(255, 69, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M28.543 15.774a2.953 2.953 0 0 0-2.951-2.949 2.882 2.882 0 0 0-1.9.713 14.075 14.075 0 0 0-6.85-2.044l1.38-4.349 3.768.884a2.452 2.452 0 1 0 .24-1.176l-4.274-1a.6.6 0 0 0-.709.4l-1.659 5.224a14.314 14.314 0 0 0-7.316 2.029 2.908 2.908 0 0 0-1.872-.681 2.942 2.942 0 0 0-1.618 5.4 5.109 5.109 0 0 0-.062.765c0 4.158 5.037 7.541 11.229 7.541s11.22-3.383 11.22-7.541a5.2 5.2 0 0 0-.053-.706 2.963 2.963 0 0 0 1.427-2.51zm-18.008 1.88a1.753 1.753 0 0 1 1.73-1.74 1.73 1.73 0 0 1 1.709 1.74 1.709 1.709 0 0 1-1.709 1.711 1.733 1.733 0 0 1-1.73-1.711zm9.565 4.968a5.573 5.573 0 0 1-4.081 1.272h-.032a5.576 5.576 0 0 1-4.087-1.272.6.6 0 0 1 .844-.854 4.5 4.5 0 0 0 3.238.927h.032a4.5 4.5 0 0 0 3.237-.927.6.6 0 1 1 .844.854zm-.331-3.256a1.726 1.726 0 1 1 1.709-1.712 1.717 1.717 0 0 1-1.712 1.712z" fill="#fff"></path></svg></span>Reddit </a><a rel="nofollow noopener" class="a2a_i" href="/#tumblr" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_tumblr" style="background-color: rgb(53, 70, 92);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.775 21.962c-.37.177-1.08.33-1.61.345-1.598.043-1.907-1.122-1.92-1.968v-6.217h4.007V11.1H17.26V6.02h-2.925s-.132.044-.144.15c-.17 1.556-.895 4.287-3.923 5.378v2.578h2.02v6.522c0 2.232 1.647 5.404 5.994 5.33 1.467-.025 3.096-.64 3.456-1.17l-.96-2.846z"></path></svg></span>Tumblr </a><a rel="nofollow noopener" class="a2a_i" href="/#wordpress" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_wordpress" style="background-color: rgb(70, 70, 70);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M3.176 16c0 5.076 2.95 9.462 7.226 11.54L4.287 10.78A12.77 12.77 0 0 0 3.177 16zm21.48-.646c0-1.586-.57-2.684-1.06-3.537-.647-1.058-1.26-1.95-1.26-3.008 0-1.18.897-2.28 2.156-2.28.057 0 .11.01.166.01A12.772 12.772 0 0 0 16 3.172c-4.48 0-8.422 2.3-10.715 5.78.302.01.585.017.826.017 1.345 0 3.42-.163 3.42-.163.69-.042.774.974.084 1.056 0 0-.694.08-1.466.12l4.668 13.892 2.808-8.417-1.998-5.476c-.69-.04-1.345-.12-1.345-.12-.69-.04-.61-1.1.08-1.058 0 0 2.117.164 3.38.164 1.34 0 3.417-.163 3.417-.163.69-.04.77.976.08 1.058 0 0-.694.08-1.467.12l4.633 13.785 1.28-4.272c.552-1.773.975-3.048.975-4.144zm-8.43 1.766l-3.85 11.18c1.15.34 2.365.523 3.624.523 1.492 0 2.925-.26 4.26-.728a1.235 1.235 0 0 1-.093-.177L16.225 17.12zM27.25 9.848c.055.408.086.848.086 1.318 0 1.3-.242 2.764-.975 4.594l-3.914 11.324C26.26 24.86 28.822 20.73 28.822 16c0-2.23-.568-4.326-1.57-6.152z"></path><path d="M16 1.052C7.757 1.052 1.052 7.757 1.052 16c0 8.242 6.705 14.948 14.948 14.948 8.242 0 14.948-6.706 14.948-14.95 0-8.24-6.706-14.946-14.948-14.946zm0 29.212c-7.865 0-14.264-6.4-14.264-14.265S8.136 1.732 16 1.732c7.863 0 14.264 6.398 14.264 14.263 0 7.863-6.4 14.264-14.264 14.264z"></path></g></svg></span>WordPress </a><a rel="nofollow noopener" class="a2a_i" href="/#google_gmail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_gmail" style="background-color: rgb(221, 83, 71);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#fff" opacity=".25" d="M4.958 6.569h22.338V25.43H4.958z"></path><path d="M28.709 7.321a1.7 1.7 0 0 0-1.409-.752h-.077l-1.1.8-.082.06-9.952 7.271L5.961 7.3l-1-.733H4.7A1.7 1.7 0 0 0 3 8.273v15.454a1.676 1.676 0 0 0 .069.481A1.7 1.7 0 0 0 4.7 25.431h1.261V11.36l7.35 5.368 2.416 1.764.445.326 2.778-2.029 7.088-5.177v13.819H27.3a1.7 1.7 0 0 0 1.634-1.223 1.675 1.675 0 0 0 .066-.481V8.273a1.7 1.7 0 0 0-.291-.952z" fill="#fff"></path></svg></span>Google Gmail </a><a rel="nofollow noopener" class="a2a_i" href="/#whatsapp" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_whatsapp" style="background-color: rgb(18, 175, 10);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill-rule="evenodd" clip-rule="evenodd" fill="#FFF" d="M16.21 4.41C9.973 4.41 4.917 9.465 4.917 15.7c0 2.134.592 4.13 1.62 5.832L4.5 27.59l6.25-2.002a11.241 11.241 0 0 0 5.46 1.404c6.234 0 11.29-5.055 11.29-11.29 0-6.237-5.056-11.292-11.29-11.292zm0 20.69c-1.91 0-3.69-.57-5.173-1.553l-3.61 1.156 1.173-3.49a9.345 9.345 0 0 1-1.79-5.512c0-5.18 4.217-9.4 9.4-9.4 5.183 0 9.397 4.22 9.397 9.4 0 5.188-4.214 9.4-9.398 9.4zm5.293-6.832c-.284-.155-1.673-.906-1.934-1.012-.265-.106-.455-.16-.658.12s-.78.91-.954 1.096c-.176.186-.345.203-.628.048-.282-.154-1.2-.494-2.264-1.517-.83-.795-1.373-1.76-1.53-2.055-.158-.295 0-.445.15-.584.134-.124.3-.326.45-.488.15-.163.203-.28.306-.47.104-.19.06-.36-.005-.506-.066-.147-.59-1.587-.81-2.173-.218-.586-.46-.498-.63-.505-.168-.007-.358-.038-.55-.045-.19-.007-.51.054-.78.332-.277.274-1.05.943-1.1 2.362-.055 1.418.926 2.826 1.064 3.023.137.2 1.874 3.272 4.76 4.537 2.888 1.264 2.9.878 3.43.85.53-.027 1.734-.633 2-1.297.266-.664.287-1.24.22-1.363-.07-.123-.26-.203-.54-.357z"></path></svg></span>WhatsApp </a><a rel="nofollow noopener" class="a2a_i" href="/#stumbleupon" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_stumbleupon" style="background-color: rgb(239, 78, 35);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.97 14.245l1.25.583 1.865-.557v-1.29A4.088 4.088 0 0 0 16 9.01a4.089 4.089 0 0 0-4.085 3.943v5.912a.969.969 0 0 1-1.937 0V16.36H6.846v2.538a4.093 4.093 0 0 0 4.092 4.093c2.242 0 4.06-1.8 4.09-4.03l.005-5.84a.97.97 0 0 1 1.94 0v1.127h-.004zm5.05 2.114v2.62a.97.97 0 1 1-1.938 0v-2.57l-1.864.555-1.25-.583v2.552a4.093 4.093 0 0 0 8.186-.036v-2.54H22.02z"></path></svg></span>StumbleUpon </a><a rel="nofollow noopener" class="a2a_i" href="/#amazon_wish_list" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_amazon" style="background-color: rgb(255, 153, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M24.998 23.842a.901.901 0 0 0-.377.086l-.4.172-.193.08-.25.1v.005c-2.72 1.102-5.573 1.748-8.215 1.805-.097.004-.193.004-.29.004-4.153.003-7.544-1.926-10.964-3.823a.803.803 0 0 0-.36-.093.646.646 0 0 0-.43.164.591.591 0 0 0-.182.427c0 .207.112.395.267.52 3.21 2.786 6.73 5.376 11.46 5.378.094 0 .188-.002.28-.004 3.01-.07 6.415-1.084 9.058-2.744l.017-.01a9.07 9.07 0 0 0 1.018-.703c.205-.15.346-.385.344-.63-.01-.435-.378-.73-.776-.73zm3.666-1.54c-.012-.265-.068-.466-.178-.632l-.01-.016-.015-.02a.863.863 0 0 0-.332-.218c-.347-.133-.853-.205-1.46-.207-.437 0-.92.04-1.4.142l-.002-.03-.486.16-.01.006-.276.09v.012a4.617 4.617 0 0 0-.89.498c-.167.13-.31.297-.317.556a.51.51 0 0 0 .185.395.6.6 0 0 0 .378.13c.027 0 .055 0 .078-.006l.023-.002.02-.003c.237-.054.585-.086.99-.145a10.3 10.3 0 0 1 1.04-.066c.226 0 .43.013.57.044a.558.558 0 0 1 .17.06c.007.02.017.067.015.14.004.268-.11.767-.266 1.25-.153.487-.34.974-.46 1.298a.66.66 0 0 0-.05.247.55.55 0 0 0 .16.392c.11.104.256.145.375.145h.006a.738.738 0 0 0 .464-.176c1.236-1.11 1.666-2.887 1.684-3.887l-.003-.16zm-11.31-11.918c-.727.055-1.564.11-2.403.222-1.28.17-2.57.39-3.63.896-2.07.838-3.466 2.627-3.466 5.254 0 3.3 2.124 4.98 4.81 4.98.894 0 1.622-.114 2.29-.28 1.064-.336 1.958-.95 3.02-2.07.614.838.782 1.23 1.844 2.125.278.115.558.115.78-.05.673-.56 1.85-1.57 2.462-2.126.28-.224.224-.56.056-.837-.613-.783-1.23-1.455-1.23-2.965V10.5c0-2.125.167-4.082-1.397-5.534-1.286-1.173-3.3-1.62-4.865-1.62h-.672c-2.85.164-5.868 1.395-6.54 4.918-.11.447.226.613.45.67l3.13.39c.336-.055.502-.336.56-.613.278-1.23 1.284-1.844 2.4-1.96h.227c.67 0 1.397.28 1.79.84.447.67.39 1.57.39 2.35v.447zm-.612 6.65c-.393.782-1.063 1.286-1.79 1.456-.112 0-.28.055-.448.055-1.228 0-1.956-.95-1.956-2.35 0-1.788 1.06-2.627 2.402-3.018.727-.167 1.567-.225 2.405-.225v.672c0 1.287.057 2.292-.613 3.41z" fill="#FFF"></path></svg></span>Amazon Wish List </a><a rel="nofollow noopener" class="a2a_i" href="/#aol_mail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_aol" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M17.334 13.26c-2.315 0-4.067 1.8-4.067 4.027 0 2.35 1.824 4.03 4.067 4.03 2.243 0 4.062-1.68 4.062-4.03 0-2.228-1.744-4.027-4.062-4.027zm0 2.127c1-.007 1.82.847 1.82 1.9 0 1.048-.82 1.9-1.82 1.9s-1.818-.853-1.818-1.9c0-1.053.817-1.9 1.818-1.9zm11.59 4.518c0 .778-.63 1.412-1.41 1.412a1.41 1.41 0 1 1 1.41-1.412zm-4.104 1.418h-2.216v-10.28h2.216v10.28zM9.33 11.04s2.585 6.79 3.862 10.13c.015.037.028.078.047.132-.06.006-.106.01-.15.01-.83.002-1.665-.003-2.498.004-.12.002-.17-.04-.204-.156-.116-.385-.247-.766-.365-1.147-.032-.11-.074-.153-.193-.153-1.066.006-2.132.006-3.2 0-.1 0-.142.03-.173.13-.127.405-.26.81-.39 1.21-.02.076-.05.117-.136.117-.873-.006-1.75-.004-2.623-.004-.016 0-.036-.005-.07-.012.023-.06.04-.115.064-.17 1.287-3.306 3.91-10.085 3.91-10.085h2.12zm-.023 6.674c-.343-1.147-.68-2.274-1.02-3.4h-.03l-1.017 3.4h2.067z" fill="#FFF"></path></svg></span>AOL Mail </a><a rel="nofollow noopener" class="a2a_i" href="/#balatarin" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_balatarin" style="background-color: rgb(7, 153, 72);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M4 16.923v9.231A1.852 1.852 0 0 0 5.846 28h20.308A1.852 1.852 0 0 0 28 26.154v-9.231zm15.692 6.462h-7.384v-1.847h7.385zM26.154 4H5.846A1.852 1.852 0 0 0 4 5.846v9.231h24V5.846A1.852 1.852 0 0 0 26.154 4zm-6.462 6.462h-2.769v2.769h-1.846v-2.769h-2.769V8.615h2.769V5.846h1.846v2.769h2.769z" fill="#fff"></path></svg></span>Balatarin </a><a rel="nofollow noopener" class="a2a_i" href="/#bibsonomy" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_bibsonomy" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M25.058 25.892c-.25 0-.484.065-.694.17l-5.907-8.2a1.548 1.548 0 0 0 .344-1.68l4.28-2.573c.288.333.71.552 1.187.552.862 0 1.56-.7 1.56-1.56 0-.776-.567-1.415-1.31-1.535V6.11a1.556 1.556 0 0 0-.25-3.095c-.862 0-1.56.7-1.56 1.56 0 .196.04.383.106.556l-4.256 2.477a1.548 1.548 0 0 0-1.2-.574c-.862 0-1.56.7-1.56 1.56 0 .778.567 1.417 1.31 1.537v5.1c-.218.035-.42.112-.598.23L9.93 8.204c.213-.268.345-.6.345-.97a1.56 1.56 0 1 0-.712 1.309l6.575 7.25c-.213.267-.345.6-.345.968 0 .23.054.45.144.647L8.08 22.66a1.547 1.547 0 0 0-1.138-.5 1.56 1.56 0 1 0 1.56 1.56c0-.23-.053-.45-.144-.646l7.857-5.25c.07.076.147.144.23.204l-1.95 3.927a1.536 1.536 0 0 0-.457-.076c-.862 0-1.56.695-1.56 1.56a1.56 1.56 0 1 0 2.468-1.27l1.95-3.926c.146.045.297.076.458.076.25 0 .484-.064.694-.17l5.907 8.2a1.56 1.56 0 1 0 1.103-.459zM18.914 8.59c0-.197-.04-.383-.106-.556l4.257-2.473c.236.29.567.49.95.55v4.96a1.558 1.558 0 0 0-1.197 2.115l-4.278 2.573a1.542 1.542 0 0 0-.935-.53v-5.102a1.555 1.555 0 0 0 1.31-1.534z"></path></svg></span>BibSonomy </a><a rel="nofollow noopener" class="a2a_i" href="/#bitty_browser" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_bitty" style="background-color: rgb(153, 153, 153);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M4 4h12v12H4z"></path><path d="M20 4v16H4v8h24V4"></path></g></svg></span>Bitty Browser </a><a rel="nofollow noopener" class="a2a_i" href="/#blinklist" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_blinklist" style="background-color: rgb(61, 60, 59);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M3 10.35v11.3l8.977-5.418"></path><path d="M17.55 27.454c-4.397 0-8.314-2.39-10.205-6.36l1.675-1.04a9.49 9.49 0 0 0 8.53 5.388c5.204 0 9.438-4.235 9.438-9.44 0-5.208-4.233-9.443-9.44-9.443a9.424 9.424 0 0 0-8.7 5.762l-1.733-1.057c1.798-4.25 5.82-6.72 10.434-6.72 6.31 0 11.45 5.138 11.45 11.45 0 6.318-5.136 11.458-11.45 11.458z"></path><path d="M11.425 18.623a6.708 6.708 0 0 0 6.18 4.09 6.713 6.713 0 0 0 6.715-6.712c0-3.71-3.005-6.71-6.714-6.71a6.71 6.71 0 0 0-6.295 4.38l3.96 2.565-3.84 2.39z"></path></g></svg></span>Blinklist </a><a rel="nofollow noopener" class="a2a_i" href="/#blogger" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_blogger" style="background-color: rgb(253, 163, 82);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M25.793 14.325l-.166-.344-.277-.214c-.363-.284-2.2.02-2.695-.43-.352-.324-.404-.905-.51-1.69-.197-1.527-.322-1.606-.56-2.122-.866-1.836-3.214-3.217-4.825-3.408h-4.367c-3.436 0-6.244 2.814-6.242 6.248v7.29c0 3.428 2.81 6.238 6.248 6.238h7.174c3.436 0 6.225-2.81 6.244-6.237l.04-5.048-.06-.277zM12.47 11.22h3.464c.66 0 1.195.534 1.195 1.188 0 .653-.54 1.195-1.2 1.195h-3.46c-.66 0-1.194-.542-1.194-1.195 0-.654.535-1.19 1.195-1.19zm7.038 9.526H12.47c-.66 0-1.194-.54-1.194-1.188 0-.654.535-1.19 1.195-1.19h7.04c.655 0 1.19.536 1.19 1.19 0 .646-.535 1.188-1.19 1.188z"></path></svg></span>Blogger </a><a rel="nofollow noopener" class="a2a_i" href="/#blogmarks" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_blogmarks" style="background-color: rgb(83, 83, 83);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M22.762 18.917L17.646 16l-8.408-4.795V29l13.524-7.71m0-10.58L9.238 3v8.204l8.408 4.794 5.116-2.915" fill="#FFF"></path></svg></span>BlogMarks </a><a rel="nofollow noopener" class="a2a_i" href="/#bookmarks_fr" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_bookmarks_fr" style="background-color: rgb(150, 192, 68);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M27.256 9.5c-2.188-3.79-6.36-.54-7.83 2.205a5.533 5.533 0 0 1 2.006 3.475c3.103.094 8.023-1.873 5.824-5.68-.7-1.212.515.894 0 0zM15.998 3c-4.368 0-3.664 5.23-2.013 7.886a5.53 5.53 0 0 1 4.023 0C19.66 8.23 20.366 3 15.998 3zM9.255 8.294c-1.108-.64-2.42-.918-3.49-.053-.96.78-1.79 2.27-1.617 3.54.352 2.563 4.32 3.467 6.416 3.404a5.527 5.527 0 0 1 2.01-3.477c-.67-1.256-1.898-2.59-3.32-3.41-.71-.41 1.422.82 0 0zm2.17 10.756a5.476 5.476 0 0 1-.86-2.23c-2.786-.084-7.77 1.608-6.057 5.24 2.023 4.292 6.448 1.248 8.063-1.765a5.427 5.427 0 0 1-1.14-1.246c-.133-.2.313.47 0 0zm6.777 2.408a6.145 6.145 0 0 0-.198-.342 5.543 5.543 0 0 1-4.02 0C12.338 23.766 11.624 29 16 29c4.218 0 3.67-4.848 2.204-7.542-.064-.12.678 1.243 0 0zm9.628-1.37c-.478-2.46-4.326-3.33-6.398-3.27a5.527 5.527 0 0 1-2.007 3.476c.934 1.744 2.858 3.73 4.913 4.006 2.043.276 3.853-2.332 3.49-4.212-.153-.8.137.706 0 0z" fill="#FFF"></path></svg></span>Bookmarks.fr </a><a rel="nofollow noopener" class="a2a_i" href="/#box_net" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_box" style="background-color: rgb(26, 116, 176);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.49 11.36c-1.653 0-3.18.524-4.434 1.41V6.543a1.617 1.617 0 0 0-3.235 0v11.903c-.017.2.002 1.37.055 1.7.53 3.73 3.73 6.604 7.61 6.604a7.693 7.693 0 0 0 7.692-7.696 7.694 7.694 0 0 0-7.695-7.695zm0 12.126a4.432 4.432 0 0 1-4.434-4.432 4.432 4.432 0 1 1 8.863 0 4.433 4.433 0 0 1-4.434 4.432z"></path></svg></span>Box.net </a><a rel="nofollow noopener" class="a2a_i" href="/#buffer" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_buffer" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M5.985 10.736l9.378 4.142c.36.122.84.107 1.157 0l9.54-4.142c.568-.2.568-.916 0-1.157L16.52 5.6c-.318-.13-.88-.135-1.157 0L5.985 9.58c-.628.265-.628.818 0 1.156zm20.076 4.57l-2.1-.877-7.44 3.23c-.395.15-.77.14-1.154 0L8.05 14.43l-2.064.875c-.628.267-.628.865 0 1.157l9.378 4.142c.364.122.753.143 1.157 0l9.545-4.142c.566-.312.566-.818 0-1.157zm0 5.796l-2.1-.876-7.44 3.23a1.55 1.55 0 0 1-1.154 0l-7.313-3.23-2.064.876c-.63.266-.63.865 0 1.156l9.377 4.142c.364.122.753.143 1.157 0l9.54-4.142c.57-.31.57-.818 0-1.156z"></path></svg></span>Buffer </a><a rel="nofollow noopener" class="a2a_i" href="/#care2_news" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_care2" style="background-color: rgb(110, 180, 63);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.56 21.21a20.225 20.225 0 0 0 4.95 2.122c.746.207 1.255.03 1.596-.673.102-.21.25-.405.4-.587 2.322-2.812 1.91-6.988-1.06-7.983.66-.517.662-1.2.47-1.92-.156-.59-.265-1.204-.492-1.767-.514-1.278-.694-2.603-.77-3.964-.047-.815-.1-1.768-1-2.07-.875-.29-1.54.358-2.09.978-1.85 2.082-2.9 4.576-3.644 7.22-.372 1.33-.7 2.676-1.077 4.12-.332-.34-.57-.837-.925-.897-.14-.375-.74-2.2.177-3.78a.11.11 0 0 0 .047-.036l.174-.22a.142.142 0 0 0-.024-.2.138.138 0 0 0-.196.025l-.174.222a.143.143 0 0 0-.02.138c-.84 1.48-.432 3.147-.23 3.76-.422-.616-1.416-1.792-2.95-2.06a.136.136 0 0 0-.116-.067h-.28a.141.141 0 1 0-.001.282h.28a.13.13 0 0 0 .062-.017c1.54.254 2.51 1.48 2.884 2.046-.38.265.128.786.08 1.276-1.985-1.483-4.07-2.605-6.243-3.572-1.956-.87-3.89-1.798-6.008-2.233-.47-.097-.978-.107-1.25.41-.25.474-.094.936.19 1.34.152.222.346.434.565.585 2.44 1.678 4.303 3.93 6.212 6.145.446.517.896 1.027 1.665 1.065.25.012.455.132.445.446v.123c.047.826.25 1.5.774 2.28 1.366 1.67 3.08 2.88 5.055 3.7.4.166.73.25 1.01.25.685 0 1.018-.518 1.116-1.58.027-1.45-.42-2.82-.862-4.153.352-.27.517.24.788.065.237-.33-.216-.68.018-1.08.172.097.316.173.456.256z"></path></svg></span>Care2 News </a><a rel="nofollow noopener" class="a2a_i" href="/#citeulike" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_citeulike" style="background-color: rgb(39, 129, 205);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M8 24h16v2H8zm0-6h16v2H8zm0-6h16v2H8zm0-6h16v2H8z" fill="#FFF"></path></svg></span>CiteULike </a><a rel="nofollow noopener" class="a2a_i" href="/#copy_link" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_link" style="background-color: rgb(1, 102, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M24.412 21.177c0-.36-.126-.665-.377-.917l-2.804-2.804a1.235 1.235 0 0 0-.913-.378c-.377 0-.7.144-.97.43.026.028.11.11.255.25.144.14.24.236.29.29s.117.14.2.256c.087.117.146.232.177.344.03.112.046.236.046.37 0 .36-.126.666-.377.918a1.25 1.25 0 0 1-.918.377 1.4 1.4 0 0 1-.373-.047 1.062 1.062 0 0 1-.345-.175 2.268 2.268 0 0 1-.256-.2 6.815 6.815 0 0 1-.29-.29c-.14-.142-.223-.23-.25-.254-.297.28-.445.607-.445.984 0 .36.126.664.377.916l2.778 2.79c.243.243.548.364.917.364.36 0 .665-.118.917-.35l1.982-1.97c.252-.25.378-.55.378-.9zm-9.477-9.504c0-.36-.126-.665-.377-.917l-2.777-2.79a1.235 1.235 0 0 0-.913-.378c-.35 0-.656.12-.917.364L7.967 9.92c-.254.252-.38.553-.38.903 0 .36.126.665.38.917l2.802 2.804c.242.243.547.364.916.364.377 0 .7-.14.97-.418-.026-.027-.11-.11-.255-.25s-.24-.235-.29-.29a2.675 2.675 0 0 1-.2-.255 1.052 1.052 0 0 1-.176-.344 1.396 1.396 0 0 1-.047-.37c0-.36.126-.662.377-.914.252-.252.557-.377.917-.377.136 0 .26.015.37.046.114.03.23.09.346.175.117.085.202.153.256.2.054.05.15.148.29.29.14.146.222.23.25.258.294-.278.442-.606.442-.983zM27 21.177c0 1.078-.382 1.99-1.146 2.736l-1.982 1.968c-.745.75-1.658 1.12-2.736 1.12-1.087 0-2.004-.38-2.75-1.143l-2.777-2.79c-.75-.747-1.12-1.66-1.12-2.737 0-1.106.392-2.046 1.183-2.818l-1.186-1.185c-.774.79-1.708 1.186-2.805 1.186-1.078 0-1.995-.376-2.75-1.13l-2.803-2.81C5.377 12.82 5 11.903 5 10.826c0-1.08.382-1.993 1.146-2.738L8.128 6.12C8.873 5.372 9.785 5 10.864 5c1.087 0 2.004.382 2.75 1.146l2.777 2.79c.75.747 1.12 1.66 1.12 2.737 0 1.105-.392 2.045-1.183 2.817l1.186 1.186c.774-.79 1.708-1.186 2.805-1.186 1.078 0 1.995.377 2.75 1.132l2.804 2.804c.754.755 1.13 1.672 1.13 2.75z"></path></svg></span>Copy Link </a><a rel="nofollow noopener" class="a2a_i" href="/#delicious" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_delicious" style="background-color: rgb(51, 153, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#fff" d="M4 4h12v12H4z"></path><path fill="#fff" opacity=".2" d="M4 16h12v12H4z"></path><path fill="#fff" opacity=".8" d="M16 16h12v12H16z"></path><path fill="#fff" opacity=".4" d="M16 4h12v12H16z"></path></svg></span>Delicious </a><a rel="nofollow noopener" class="a2a_i" href="/#design_float" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_designfloat" style="background-color: rgb(138, 200, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M16 3C8.814 3 3 8.814 3 16s5.814 13 13 13 13-5.814 13-13S23.187 3 16 3zm0 25.152c-6.712 0-12.153-5.44-12.153-12.152C3.847 9.288 9.287 3.848 16 3.848S28.152 9.288 28.152 16c0 6.712-5.44 12.152-12.152 12.152z"></path><path d="M22.406 16A6.402 6.402 0 0 0 16 9.593 6.402 6.402 0 0 0 9.593 16 6.4 6.4 0 0 0 16 22.406 6.4 6.4 0 0 0 22.406 16zM16 21.39A5.392 5.392 0 0 1 10.61 16 5.403 5.403 0 0 1 16 10.61 5.393 5.393 0 0 1 21.39 16 5.382 5.382 0 0 1 16 21.39z"></path><path d="M13.763 9.187V4.864c-4.475.9-8 4.424-8.898 8.898h4.322a7.226 7.226 0 0 1 4.576-4.575zm9.05 4.576h4.32c-.896-4.475-4.422-8-8.896-8.898v4.322a7.224 7.224 0 0 1 4.575 4.576zm-4.576 9.052v4.322c4.475-.9 8-4.424 8.897-8.9h-4.322a7.232 7.232 0 0 1-4.575 4.578zm-9.05-4.578H4.863c.898 4.475 4.424 8 8.898 8.9v-4.323a7.233 7.233 0 0 1-4.574-4.577z"></path></g></svg></span>Design Float </a><a rel="nofollow noopener" class="a2a_i" href="/#diary_ru" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_diary_ru" style="background-color: rgb(145, 45, 49);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 3C8.82 3 3 8.82 3 16s5.82 13 13 13 13-5.82 13-13S23.18 3 16 3zm0 24.807C9.48 27.807 4.192 22.522 4.192 16 4.192 9.48 9.48 4.193 16 4.193c3.92 0 7.392 1.91 9.54 4.85h-8.308s-2.863.397-3.18 2.544c-.34 2.293-1.988 2.465-1.988 2.465h-4.69v1.51h9.74a2.343 2.343 0 1 1 4.605.001h1.633v.874H21.72a2.348 2.348 0 0 1-2.304 1.91c-1.147 0-2.1-.823-2.306-1.91H7.37v1.59h4.69s1.67 0 1.988 2.464c.304 2.357 3.18 2.55 3.18 2.55h8.25a11.786 11.786 0 0 1-9.48 4.77z"></path></svg></span>Diary.Ru </a><a rel="nofollow noopener" class="a2a_i" href="/#diaspora" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_diaspora" style="background-color: rgb(46, 52, 54);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M13.498 6.49v6.258l-5.953-1.933L6 15.57l5.95 1.934-3.677 5.063 4.046 2.942L16 20.44l3.68 5.064 4.047-2.943L20.05 17.5 26 15.57l-1.545-4.755-5.953 1.933V6.49h-5.004z"></path></svg></span>Diaspora </a><a rel="nofollow noopener" class="a2a_i" href="/#digg" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_digg" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M7.07 8.122V11.8h-4.2v8.927h6.828V8.122H7.07zm0 10.504H5.497V13.9H7.07v4.726zm3.678-6.826h2.626v8.927h-2.626zm0-3.678h2.626v2.626h-2.626zm3.677 3.678v8.927h4.2v1.05h-4.2v2.102h6.827V11.8h-6.827zm4.2 6.826H17.05V13.9h1.577v4.726zm3.677-6.826v8.927h4.202v1.05h-4.202v2.102h6.828V11.8h-6.828zm4.202 6.826H24.93V13.9h1.574v4.726z" fill="#FFF"></path></svg></span>Digg </a><a rel="nofollow noopener" class="a2a_i" href="/#diigo" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_diigo" style="background-color: rgb(74, 139, 202);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill-rule="evenodd" clip-rule="evenodd" fill="#FFF" d="M23.81 4.5c.012.198.035.396.035.593 0 4.807.026 9.615-.01 14.422-.02 3.248-1.5 5.678-4.393 7.158-4.66 2.385-10.495-.64-11.212-5.836-.76-5.517 3.747-9.56 8.682-9.018 1.114.12 2.16.5 3.134 1.07.517.3.527.294.53-.29.007-2.7.01-5.4.014-8.104h3.22zm-7.914 19.97c2.608.068 4.82-2.025 4.954-4.552.138-2.626-1.89-5.074-4.727-5.145-2.7-.067-4.867 2-4.973 4.71-.107 2.72 2.13 5.008 4.746 4.988z"></path></svg></span>Diigo </a><a rel="nofollow noopener" class="a2a_i" href="/#douban" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_douban" style="background-color: rgb(0, 119, 17);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M5.755 5.505h20.55v2.59H5.755v-2.59zm14.99 18.47l1.93-4.29h2.21v-9.97h-18v9.98h2.04l1.93 4.29h-5.78v2.51h21.85v-2.51l-6.18-.01zm-10.74-6.95v-4.88h11.68v4.88h-11.68zm7.99 6.95h-4.37l-1.93-4.29h8.24l-1.94 4.29z"></path></svg></span>Douban </a><a rel="nofollow noopener" class="a2a_i" href="/#draugiem" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_draugiem" style="background-color: rgb(255, 102, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M21.55 11.33c4.656.062 7.374 2.92 4.294 6.828-1.415 1.798-3.812 3.575-7.003 4.725-.15.056-.303.105-.46.16-.3.098-.595.188-.89.28a24.866 24.866 0 0 1-4.05.814c-.464.043-.91.078-1.35.085-2.97.077-5.205-.74-5.93-2.474-.88-2.077.9-4.976 4.454-7.178-2.627 1.06-7.408 3.546-7.61 7.12v.454c.02.362.09.725.21 1.108.76 2.41 4.333 3.533 8.884 3.13.446-.036.892-.092 1.352-.16.66-.1 1.337-.23 2.027-.39a35.76 35.76 0 0 0 2.02-.558c.154-.056.3-.098.454-.153.31-.094.608-.2.9-.31 3.945-1.436 6.87-3.34 8.58-5.526.975-1.253 1.476-2.424 1.574-3.448v-.787c-.28-2.61-3.317-4.135-7.45-3.717zm-3.024-1.29c.11 0 .21-.014.307-.035.662-.167.983-.87 1.01-1.7.028-.885-.286-1.624-1.01-1.728-.063-.014-.125-.014-.195-.014-.578 0-.955.348-1.157.857-.094.265-.16.564-.163.885-.014.383.034.745.167 1.038.196.418.53.697 1.046.697zm-.014.292c-.293 0-.544.028-.76.084l.063.084.11.202.092.21.077.215.056.223.035.223.02.23.008.223v.237l-.014.23-.018.23-.028.23-.028.23-.043.23-.042.23-.04.223-.056.223-.042.212-.056.21-.057.2-.057.196-.042.19-.04.18-.02.11-.03.125-.028.132-.02.14-.03.152-.02.124v.03l-.028.166-.056.21-.02.172-.03.18-.02.182-.03.18-.02.19-.03.18-.02.188-.02.188-.02.19v.007c.04.537.082.997.103 1.26.02.3.085.517.18.663.14.215.378.292.706.32.28-.028.487-.084.647-.23.153-.14.237-.376.3-.753.118-.774.467-3.31.767-4.397.425-1.568 1.456-4.418-1.066-4.634-.122-.024-.226-.024-.338-.024zm-3.06-.8h.015c.976-.008 1.436-.9 1.436-1.994s-.46-1.993-1.436-2h-.014c-.99 0-1.45.9-1.45 2s.46 1.993 1.45 1.993zm-2.013 4.626c.09.383.18.732.254 1.052.307 1.254.606 4.16.718 5.038.105.885.418 1.073 1.052 1.136.62-.063.94-.25 1.045-1.136.105-.878.41-3.79.71-5.038.08-.314.175-.67.266-1.052.28-1.15.502-2.495 0-3.366-.32-.557-.94-.92-2.02-.92-1.088 0-1.708.37-2.03.92-.5.864-.27 2.216 0 3.366zm-1.35-4.153c.1.02.196.035.308.035.516 0 .857-.28 1.045-.704.118-.293.174-.655.167-1.038a2.96 2.96 0 0 0-.167-.885c-.202-.51-.585-.857-1.157-.857-.07 0-.134 0-.197.014-.725.105-1.045.843-1.01 1.728.02.836.35 1.54 1.01 1.707zm-.3 9.373c.057.376.154.606.3.753.16.157.37.206.65.23.33-.024.557-.1.704-.32.09-.14.153-.36.18-.66.022-.264.064-.72.106-1.253v-.014l-.02-.187-.02-.188-.03-.188-.02-.18-.02-.19-.03-.18-.02-.18-.03-.183-.025-.174-.02-.166-.03-.167v-.02l-.02-.133-.028-.153-.028-.14-.024-.13-.028-.125-.03-.11-.034-.184-.056-.188-.04-.196-.058-.203-.056-.21-.056-.215-.04-.223-.057-.225-.04-.23-.033-.23-.028-.23-.03-.23-.02-.23-.008-.237v-.23l.007-.223.02-.23.034-.223.056-.222.07-.216.1-.21.11-.2.065-.085a3.128 3.128 0 0 0-.76-.083c-.11 0-.216 0-.32.014-2.524.216-1.492 3.066-1.067 4.634.262 1.054.603 3.59.728 4.364z"></path></svg></span>Draugiem </a><a rel="nofollow noopener" class="a2a_i" href="/#dzone" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_dzone" style="background-color: rgb(130, 194, 81);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M16.957 21.422l8.6-10.75h-7.972V9.008H28v1.628l-8.65 10.69H28v1.667H16.957v-1.57zM4 9.007h5.59c1.898 0 3.37.683 4.416 2.047.933 1.23 1.4 2.81 1.4 4.73 0 1.486-.277 2.83-.828 4.028-.97 2.12-2.64 3.18-5.007 3.18H4V9.007zm5.22 12.367c.626 0 1.14-.066 1.544-.2.72-.247 1.31-.724 1.768-1.428.367-.565.63-1.29.792-2.17.094-.527.14-1.016.14-1.466 0-1.732-.336-3.078-1.01-4.037-.674-.958-1.758-1.438-3.254-1.438H5.913v10.74H9.22z" fill="#FFF"></path></svg></span>DZone </a><a rel="nofollow noopener" class="a2a_i" href="/#evernote" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_evernote" style="background-color: rgb(139, 224, 86);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M7.884 8.573h2.276c.13 0 .236-.106.236-.235 0 0-.027-1.95-.027-2.494v-.006c0-.445.09-.833.252-1.16l.078-.145c-.007 0-.017.005-.025.014l-4.42 4.385a.044.044 0 0 0-.017.026c.09-.046.215-.107.233-.115.387-.175.85-.27 1.41-.27zm17.704-.477c-.18-.968-.755-1.444-1.275-1.632-.56-.203-1.698-.413-3.127-.58-1.15-.137-2.504-.126-3.318-.1-.1-.672-.568-1.285-1.096-1.498-1.404-.564-3.573-.428-4.13-.272-.442.125-.932.378-1.205.768-.183.262-.302.595-.302 1.062 0 .265.007.886.015 1.44l.014 1.054c0 .494-.4.896-.896.897H7.99c-.485 0-.856.082-1.14.21a1.572 1.572 0 0 0-.636.508c-.304.408-.357.912-.355 1.426 0 0 0 .415.1 1.23.085.63.768 5.02 1.415 6.355.25.522.42.736.912.966 1.1.47 3.61.995 4.787 1.147 1.174.15 1.912.466 2.35-.457.002 0 .088-.227.208-.56.382-1.156.435-2.18.435-2.924 0-.076.11-.078.11 0 0 .524-.1 2.38 1.303 2.875.554.198 1.7.374 2.864.51 1.055.12 1.82.538 1.82 3.24 0 1.646-.346 1.87-2.152 1.87-1.464 0-2.02.04-2.02-1.124 0-.938.93-.842 1.616-.842.31 0 .086-.23.086-.81 0-.576.36-.91.02-.918-2.384-.065-3.786-.004-3.786 2.978 0 2.706 1.036 3.208 4.418 3.208 2.65 0 3.588-.086 4.682-3.483.22-.67.742-2.72 1.06-6.155.197-2.173-.194-8.732-.502-10.388zm-4.622 7.25a4.915 4.915 0 0 0-.937.056c.08-.667.352-1.488 1.33-1.453 1.08.032 1.23 1.055 1.238 1.75a4.402 4.402 0 0 0-1.635-.358z"></path></svg></span>Evernote </a><a rel="nofollow noopener" class="a2a_i" href="/#facebook_messenger" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_facebook_messenger" style="background-color: rgb(0, 132, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 5C9.986 5 5.11 9.56 5.11 15.182c0 3.2 1.58 6.054 4.046 7.92V27l3.716-2.06c.99.276 2.04.425 3.128.425 6.014 0 10.89-4.56 10.89-10.183S22.013 5 16 5zm1.147 13.655L14.33 15.73l-5.423 3 5.946-6.31 2.816 2.925 5.42-3-5.946 6.31z"></path></svg></span>Facebook Messenger </a><a rel="nofollow noopener" class="a2a_i" href="/#fark" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_fark" style="background-color: rgb(85, 85, 85);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M22.925 9.77V5.613H9.075v20.772h5.54v-8.31h8.31v-4.153h-8.31V9.77"></path></svg></span>Fark </a><a rel="nofollow noopener" class="a2a_i" href="/#flipboard" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_flipboard" style="background-color: rgb(204, 0, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M19 19H7V7h12v12z"></path><path d="M25 13H7V7h18v6z"></path><path d="M13 25H7V7h6v18z"></path></g></svg></span>Flipboard </a><a rel="nofollow noopener" class="a2a_i" href="/#folkd" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_folkd" style="background-color: rgb(15, 112, 178);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M19.956 10.21c-.183.192-.613.138-.99.14-.102.317-.126.71-.283.974.626.496 1.85.406 2.405.972 1.197-.444.245-1.354.99-2.085 1.134-.14 1.386.59 1.697 1.255-.267.247-.936.102-1.415.14-.26.208-.333.6-.565.833.258.58.96.724 1.133 1.39a52.822 52.822 0 0 1 4.526-.973c.41-.43.317-1.356.99-1.53 1.104.213 1.46 1.16 1.556 2.363-.586 1.043-2.1.605-2.546-.277-1.465.227-2.867.52-4.245.833-.11.325.1 1.263-.145 1.81.926.625 1.29-.49 2.122-.42.48.503.495 2.374-.566 2.224-.603-.053-.673-.634-.707-1.25-.432.1-.515-.143-.85-.14-.646.383-.97 1.083-1.695 1.39.094 1.236 1.462.38 2.12.974-.057 1.01-.8 1.346-1.838 1.39-.066-.835.216-1.502-.707-1.944-.94.142-1.37.782-2.264.973-.014.43.174.663.283.97.248.17 1.165-.21 1.415.28-.01 1.074-1.473 1.827-2.264 1.25-.063-.617.382-.735.565-1.11-.268-.294-.406-.714-.565-1.113-.878.342-1.665.773-2.83.834.004.328-.243.41-.14.834-.056.518.68.26.706.696.24 1.26-1.776 1.455-2.12.555-.03-.86 1.034-.65.707-1.808-.713-.274-1.917-.063-2.55-.417-.374.14-.382.644-.706.835.066.4.5.436.426.974-.925.36-2.313.07-2.12-1.115.533-.615 1.49-.076 1.835-.973-.556-.656-1.165-1.263-1.412-2.223-2.124.463-3.843 1.322-5.8 1.946-.187.467-.196 1.107-.566 1.39-.59.148-.625.036-1.273 0-.56-.854-.258-2.7.85-2.78.61-.135.41.523.85.557 2.015-.472 3.78-1.195 5.8-1.667-.434-2.485 3.205-4.233-.85-3.893-.397-1.225.372-2.164 1.13-2.085 1.272.132.284 1.725 1.272 2.224.908-.498 1.543-1.263 2.69-1.53.08-.45-.097-.646-.143-.972-.09-.33-.87.02-.99-.28v-.97c.46-.426 1.662-.355 1.98.138.113.623-.25.773-.707.834.037.427.308.626.424.972.9-.37 2.268-.273 3.538-.277.2-.17.266-.48.28-.836.108-.52-.75-.094-.564-.693.157-1.48 2.126-.633 2.12.416zm1.132 6.256c.652-.378.368-1.677.424-2.64-1.742-1.426-2.885 2.7-.424 2.64zm-6.224 1.53c1.3.173 3.114-1.23 2.97-2.502-.14-1.227-2.55-2.332-3.96-1.67-1.815.858-.44 3.98.99 4.17z"></path></svg></span>Folkd </a><a rel="nofollow noopener" class="a2a_i" href="/#google_bookmarks" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_google" style="background-color: rgb(66, 133, 244);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.194 17.94v-3.796h9.55c.144.643.256 1.244.256 2.09 0 5.827-3.91 9.97-9.796 9.97C10.572 26.204 6 21.632 6 16S10.572 5.796 16.204 5.796c2.755 0 5.06 1.01 6.827 2.663l-2.894 2.812c-.734-.694-2.02-1.51-3.93-1.51-3.376 0-6.13 2.805-6.13 6.243s2.754 6.244 6.13 6.244c3.91 0 5.348-2.704 5.613-4.306h-5.623v-.007z"></path></svg></span>Google Bookmarks </a><a rel="nofollow noopener" class="a2a_i" href="/#google_classroom" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_google_classroom" style="background-color: rgb(255, 193, 18);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M22.667 16.667a1.667 1.667 0 1 0 0-3.334 1.667 1.667 0 0 0 0 3.334zM22.333 18c-1.928 0-4 .946-4 2.117v1.217h8v-1.217c0-1.17-2.072-2.117-4-2.117zm-13-1.333a1.668 1.668 0 1 0-.002-3.336 1.668 1.668 0 0 0 .002 3.336zM9.667 18c-1.928 0-4 .946-4 2.117v1.217h8v-1.217c0-1.17-2.072-2.117-4-2.117z"></path><path d="M15.335 15.333A2.332 2.332 0 1 0 13 13a2.333 2.333 0 0 0 2.335 2.333zm.332 1.334c-2.572 0-5.333 1.392-5.333 3.11v1.557H21v-1.556c0-1.72-2.762-3.11-5.333-3.11zm3 10.666h8v2h-8v-2z"></path></g></svg></span>Google Classroom </a><a rel="nofollow noopener" class="a2a_i" href="/#hacker_news" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_y18" style="background-color: rgb(255, 102, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M17.275 17.834v7.13h-2.602v-7.182L9 7.035h3.07l2.967 6.115c.365.755.702 1.51.988 2.316.312-.728.65-1.483 1.042-2.29l3.018-6.142H23l-5.725 10.8z"></path></svg></span>Hacker News </a><a rel="nofollow noopener" class="a2a_i" href="/#hatena" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_hatena" style="background-color: rgb(0, 166, 219);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M6.96 8.33h7.328c1.247 0 2.206.366 2.875 1.098.666.733 1.002 1.64 1.002 2.72 0 .91-.24 1.688-.715 2.336-.318.433-.784.773-1.396 1.023.928.266 1.614.72 2.05 1.367.44.645.66 1.457.66 2.432 0 .795-.157 1.512-.468 2.146-.314.635-.74 1.14-1.28 1.508-.337.23-.842.396-1.52.502-.9.14-1.498.21-1.79.21H6.958V8.328zm3.877 6.017h1.74c.623 0 1.058-.13 1.302-.382.24-.255.362-.623.362-1.104 0-.44-.123-.79-.366-1.043-.245-.25-.67-.377-1.276-.377h-1.766v2.91zm0 6.027h2.038c.69 0 1.176-.145 1.458-.434.282-.29.425-.68.425-1.168 0-.453-.142-.818-.42-1.092-.28-.277-.77-.414-1.47-.414h-2.03v3.108zM21.213 8.52h3.584v9.58h-3.584z"></path><circle cx="23.005" cy="21.635" r="2.036"></circle></g></svg></span>Hatena </a><a rel="nofollow noopener" class="a2a_i" href="/#houzz" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_houzz" style="background-color: rgb(122, 193, 67);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 4L9.072 7.999v8.002L16 12V4"></path><path fill="#FFF" d="M9.072 16v8L16 19.999 9.072 16M16 19.999V28l6.928-4v-8L16 19.999m0-7.998L22.928 16V7.999L16 12.001"></path><path opacity=".5" fill="#FFF" d="M9.072 16L16 19.999v-7.998L9.072 16"></path></svg></span>Houzz </a><a rel="nofollow noopener" class="a2a_i" href="/#instapaper" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_instapaper" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M11.98 5.5h8.04v1.265h-.62c-.582 0-1.004.08-1.264.242-.262.162-.453.39-.572.69-.12.297-.182.874-.182 1.732v13.53c0 .68.064 1.165.195 1.45.13.287.313.495.55.626.234.13.658.196 1.27.196h.618V26.5H11.98v-1.265h.662c.592 0 1.012-.067 1.258-.203.246-.135.424-.33.533-.587.11-.256.166-.75.166-1.483V9.112c0-.776-.06-1.3-.17-1.567a1.108 1.108 0 0 0-.533-.59c-.247-.128-.667-.19-1.26-.19h-.66V5.5z"></path></svg></span>Instapaper </a><a rel="nofollow noopener" class="a2a_i" href="/#kakao" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_kakao" style="background-color: rgb(252, 183, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.345 6h-8.688c-.583 0-1.06.45-1.06 1.005v8.814c0 .55.477 1 1.06 1h4.007c-.03.98-.445 2.057-1.077 2.997-.612.904-1.613 1.796-2.156 2.223l-.04.033c-.115.107-.2.23-.202.405-.003.13.07.232.15.34l.018.022 2.774 2.975s.136.137.246.163c.126.03.27.032.368-.042 4.84-3.56 5.537-8.023 5.66-10.44V7.004C21.404 6.45 20.93 6 20.346 6"></path></svg></span>Kakao </a><a rel="nofollow noopener" class="a2a_i" href="/#kik" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_kik" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M23.436 20.74a2.511 2.511 0 0 0 .109-5.019h-.11a2.516 2.516 0 0 0-2.507 2.515 2.509 2.509 0 0 0 2.508 2.508zm-7.946-3.09l2.89-2.89c.93-.93.93-2.434 0-3.363a2.374 2.374 0 0 0-3.362 0l-4.262 4.263V7.267A2.378 2.378 0 0 0 6 7.263V24.7a2.378 2.378 0 0 0 4.756.002v-2.316l1.335-1.335 3.76 5.07a2.378 2.378 0 0 0 3.866-2.771c-.016-.02-.03-.04-.047-.06l-4.177-5.638v-.002z" fill="#FFF"></path></svg></span>Kik </a><a rel="nofollow noopener" class="a2a_i" href="/#kindle_it" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_kindle" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M12.927 27H10V5h2.927v11.754l5.15-5.47h3.683l-5.814 6.067L22 27h-3.407l-4.704-7.763-.966 1.037V27z"></path></svg></span>Kindle It </a><a rel="nofollow noopener" class="a2a_i" href="/#known" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_known" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M16.38 2.65c-7.45 0-13.5 6.048-13.5 13.5s6.05 13.5 13.5 13.5 13.5-6.048 13.5-13.5-6.04-13.5-13.5-13.5zm.078 25.203c-6.387 0-11.57-5.184-11.57-11.57 0-6.388 5.183-11.572 11.57-11.572S28.03 9.898 28.03 16.28c0 6.39-5.176 11.573-11.572 11.573z"></path><path d="M23.856 21.758c-.393 0-.694-.07-.903-.2-.154-.094-.4-.402-.74-.91l-3.934-5.964 3.3-3.2c.252-.248.46-.433.623-.54.162-.11.293-.178.41-.217.107-.03.307-.046.593-.046h.207v-.01l.555-.01V8.55h-.65v.01h-6.086v2.12h.548c.31 0 .494.023.57.077.078.054.117.13.117.23 0 .055-.024.11-.063.178-.04.07-.154.2-.34.4l-4.257 4.437v-4.08c0-.402.03-.68.1-.826a.698.698 0 0 1 .34-.34c.1-.053.363-.076.78-.076h.44V8.56H8.8v2.113h.563c.34 0 .58.04.71.116.132.073.225.19.286.343.06.154.082.455.082.91v8.37c0 .478-.022.78-.076.903a.615.615 0 0 1-.285.33c-.13.07-.394.108-.78.108h-.5v2.12h6.666v-2.12h-.556c-.363 0-.61-.03-.733-.093a.566.566 0 0 1-.278-.31c-.065-.144-.1-.407-.1-.785V19.02l2.065-2.012 2.4 3.84c.2.333.3.54.3.626 0 .077-.054.147-.162.2-.108.055-.417.078-.918.078h-.278v2.12h6.89v-2.12h-.24z"></path></g></svg></span>Known </a><a rel="nofollow noopener" class="a2a_i" href="/#line" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_line" style="background-color: rgb(0, 195, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M28 14.304c0-5.37-5.384-9.738-12-9.738S4 8.936 4 14.304c0 4.814 4.27 8.846 10.035 9.608.39.084.923.258 1.058.592.122.303.08.778.04 1.084l-.172 1.028c-.05.303-.24 1.187 1.04.647s6.91-4.07 9.43-6.968c1.74-1.905 2.57-3.842 2.57-5.99zM11.302 17.5H8.918a.631.631 0 0 1-.63-.63V12.1a.63.63 0 0 1 1.26.002v4.14h1.754c.35 0 .63.28.63.628a.63.63 0 0 1-.63.63zm2.467-.63a.631.631 0 0 1-.63.628.629.629 0 0 1-.63-.63V12.1a.63.63 0 1 1 1.26 0v4.77zm5.74 0a.632.632 0 0 1-1.137.378l-2.443-3.33v2.95a.631.631 0 0 1-1.26 0V12.1a.634.634 0 0 1 .63-.63.63.63 0 0 1 .504.252l2.444 3.328V12.1a.63.63 0 0 1 1.26 0v4.77zm3.853-3.014a.63.63 0 1 1 0 1.258H21.61v1.126h1.755a.63.63 0 1 1 0 1.258H20.98a.63.63 0 0 1-.628-.63V12.1a.63.63 0 0 1 .63-.628h2.384a.63.63 0 0 1 0 1.258h-1.754v1.126h1.754z"></path></svg></span>Line </a><a rel="nofollow noopener" class="a2a_i" href="/#livejournal" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_livejournal" style="background-color: rgb(17, 49, 64);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M16.76 5.53c-1.586 0-3.094.33-4.462.922l-2.47-2.465h-.005A11.268 11.268 0 0 0 4 9.843l2.472 2.47a11.283 11.283 0 0 1 5.823-5.86h.004l9.14 9.128h-.006a11.304 11.304 0 0 0-5.823 5.86l-9.14-9.13c-.597 1.372-.952 2.865-.952 4.457 0 6.21 5.03 11.24 11.24 11.24S28 22.98 28 16.77c0-6.206-5.033-11.24-11.24-11.24zm1.95 16.573a6.561 6.561 0 0 1 3.38-3.403l.887 4.285-4.268-.882z"></path><path d="M22.092 18.702l-.653-3.118h-.007a11.288 11.288 0 0 0-5.822 5.858l3.097.66a6.58 6.58 0 0 1 3.38-3.4"></path></g></svg></span>LiveJournal </a><a rel="nofollow noopener" class="a2a_i" href="/#mail_ru" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_mail_ru" style="background-color: rgb(53, 111, 172);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M19.975 15.894c-.134-2.542-2.02-4.07-4.3-4.07h-.086c-2.63 0-4.09 2.068-4.09 4.417 0 2.635 1.763 4.298 4.075 4.298 2.58 0 4.275-1.89 4.4-4.127l-.003-.514zm-4.372-6.346c1.756 0 3.408.776 4.62 1.993v.008c0-.584.396-1.024.94-1.024h.14c.852 0 1.026.808 1.026 1.063v9.08c-.06.595.614.9.99.52 1.457-1.497 3.203-7.702-.907-11.295-3.83-3.35-8.967-2.8-11.7-.915-2.904 2-4.764 6.436-2.957 10.6 1.968 4.544 7.6 5.897 10.947 4.547 1.696-.683 2.48 1.608.72 2.356-2.66 1.132-10.065 1.02-13.524-4.972-2.34-4.046-2.214-11.163 3.986-14.85 4.74-2.822 10.99-2.042 14.762 1.895 3.936 4.116 3.704 11.82-.138 14.817-1.742 1.36-4.326.035-4.312-1.947l-.02-.65a6.456 6.456 0 0 1-4.58 1.906c-3.475 0-6.53-3.057-6.53-6.53 0-3.507 3.057-6.6 6.53-6.6"></path></svg></span>Mail.Ru </a><a rel="nofollow noopener" class="a2a_i" href="/#mendeley" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_mendeley" style="background-color: rgb(167, 8, 5);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M26.596 18.11c-1.466-.087-2.02-.834-1.93-2.164.076-1.113.274-2.22.418-3.327-.023-1.745-.942-3.13-2.488-3.59-1.583-.47-2.97-.14-4.102 1.15-2.322 2.644-2.616 2.632-5.023-.047-1.15-1.28-2.85-1.66-4.39-.98-1.5.667-2.37 2.237-2.15 3.954.08.625.28 1.235.38 1.863.337 2.123-.107 2.7-2.227 3.148-1.066.228-1.913.786-2.05 1.99-.137 1.22.17 2.39 1.404 2.75.77.226 1.854.084 2.55-.32.96-.553 1.064-1.64.733-2.74-.62-2.05-.028-3.04 2.114-3.34a5.973 5.973 0 0 1 2.568.235c1.302.41 1.692 1.373 1.175 2.65-.45 1.1-.443 2.09.39 2.984.84.9 2.417 1.08 3.518.435 1.12-.657 1.497-1.807 1.042-3.164-.608-1.814-.085-2.783 1.807-3.123a5.94 5.94 0 0 1 2.16.025c1.834.367 2.377 1.377 1.84 3.188-.504 1.698.196 3.09 1.72 3.43 1.332.295 2.624-.607 2.89-2.022.308-1.633-.593-2.882-2.344-2.988zm-10.71-.085a2.556 2.556 0 0 1-2.445-2.57c.01-1.46 1.15-2.567 2.61-2.54 1.47.026 2.57 1.177 2.52 2.627a2.568 2.568 0 0 1-2.682 2.483z"></path></svg></span>Mendeley </a><a rel="nofollow noopener" class="a2a_i" href="/#meneame" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_meneame" style="background-color: rgb(255, 125, 18);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M27.03 7.804c-.012-.44-.617-1.916-.893-1.916l-.067.01-.03.04c-.043.055-.07.157.083.714.23 1.14.413 2.764-.68 3.712-.658.667-1.562 1.052-2.437 1.425-1.208.51-2.456 1.042-3.09 2.334-.76 2.156.583 5.37 1.563 7.716.203.496.4.963.56 1.38.044.09.05.184.01.28-.224.58-1.725 1.16-2.82 1.584-.977.378-1.316.52-1.32.704 0 .037.01.11.112.155l.02.01h.023c2.303-.027 3.968-.564 4.688-1.514.372-.49.48-1.08.312-1.708-.144-.536-.448-1.233-.8-2.04-.7-1.61-1.575-3.616-1.575-5.315.045-1.5 1.388-2.156 2.687-2.787.313-.15.61-.295.874-.445 1.64-.734 3.208-2.413 2.78-4.344z"></path><path d="M15.32 25.348c-.423-.03-.82-.057-1.108-.1-.32-.035-.646-.067-.974-.1-1.927-.19-3.92-.387-5.528-1.5-1.79-1.34-1.916-3.96-1.124-5.748.644-1.635 1.616-3.337 2.89-5.06l.066-.09c.07.21.15.42.247.622.812 1.687 1.77 2.585 2.92 2.745.31.046.62.07.925.07 1.04 0 1.977-.267 2.716-.77l.38-.26-.452.08c-.168.028-.34.06-.51.094-.535.106-1.086.21-1.617.21-.617 0-1.12-.15-1.54-.46-1.882-1.203-2.623-3.85-1.582-5.67.743-1.402 2.184-2.104 4.536-2.21 2.03-.092 3.998.83 5.16 1.37.292.136.545.253.74.326.482.152 1.04.314 1.577.314.62 0 1.083-.226 1.374-.66.27-.32.3-1.006.176-1.484-.046-.178-.135-.402-.295-.462l-.15-.056.016.16c.048.444-.11.938-.397 1.256a.96.96 0 0 1-.714.34c-.09 0-.187-.014-.29-.043-.53-.124-1.044-.35-1.54-.57-.18-.08-.363-.16-.54-.232-1.76-.91-3.38-1.352-4.944-1.352-.61 0-1.22.07-1.805.207-1.19.22-2.786.9-3.693 2.137-.9 1.074-1.16 2.65-.738 4.122l-.16.132-.467.38c-.704.563-1.367 1.096-2.01 2.02-1.097 1.726-2.3 4.013-1.868 6.39.404 2.53 2.847 4.146 5.05 4.434 1.068.125 2.18.185 3.4.185h.002c1.088 0 2.25-.045 3.66-.147l.037-.002.026-.024c.09-.09.08-.17.065-.212-.082-.224-.645-.286-1.91-.375z"></path><path d="M18.398 21.133c-.044-.124-.178-.502-2.283-.502-1.357 0-3.012.17-3.363.3l-.427.16.454.034c.313.025.68.037 1.063.05 1.257.04 2.82.088 3.634.644.18.316.053 1.464-.05 2.39-.11 1.004-.17 1.582-.044 1.77l.02.03.034.01a.279.279 0 0 0 .063.008c.095 0 .184-.06.267-.177.515-.734.933-3.86.627-4.713z"></path></g></svg></span>Meneame </a><a rel="nofollow noopener" class="a2a_i" href="/#mixi" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_mixi" style="background-color: rgb(209, 173, 90);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.09 5.246C9.617 5.246 4 9.216 4 16.63c0 6.93 7.707 10.193 12.758 9.01v2.374S28 25.054 28 15.034c0-6.11-4.505-9.788-11.91-9.788zm6.83 14.778h-1.657v-5.688s-.505-1.586-1.585-1.586c-.9 0-2.525.374-2.525 2.08v5.193h-1.657V14.77c0-1.586-.787-2.09-1.506-2.09-1.15 0-2.727.807-2.727 2.403v4.94H9.606v-9.01h1.657v1.03c.656-.546 1.564-1.03 2.727-1.03 1.224 0 2.092.434 2.606 1.282a4.484 4.484 0 0 1 3.082-1.202c1.97 0 3.24 1.788 3.24 3.202v5.73z"></path></svg></span>Mixi </a><a rel="nofollow noopener" class="a2a_i" href="/#myspace" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_myspace" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M24 17.716c-2.21 0-4 1.79-4 4v1.712h8v-1.713c0-2.21-1.79-4-4-4z"></path><circle cx="24" cy="12.572" r="4"></circle><path d="M15.148 18.31a3.715 3.715 0 0 0-3.72 3.71v1.408h7.437c.002-.615.002-1.148.002-1.408 0-2.05-1.665-3.71-3.72-3.71z"></path><ellipse cx="15.147" cy="13.447" rx="3.719" ry="3.71"></ellipse><path d="M7.148 18.875A3.14 3.14 0 0 0 4 22.008v1.42h6.295c.002-.636.002-1.178.002-1.42 0-1.73-1.41-3.133-3.15-3.133z"></path><ellipse cx="7.148" cy="14.58" rx="3.148" ry="3.133"></ellipse></g></svg></span>MySpace </a><a rel="nofollow noopener" class="a2a_i" href="/#netvouz" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_netvouz" style="background-color: rgb(102, 204, 51);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M10.25 8.72v17.184H5.5V6.096h8.396l5.605 5.77v6.43"></path><path d="M21.75 23.28V6.095h4.75v19.808h-8.396L12.5 20.13v-6.427"></path></g></svg></span>Netvouz </a><a rel="nofollow noopener" class="a2a_i" href="/#odnoklassniki" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_odnoklassniki" style="background-color: rgb(242, 114, 12);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 16.16a6.579 6.579 0 0 1-6.58-6.58A6.578 6.578 0 0 1 16 3a6.58 6.58 0 1 1 .002 13.16zm0-9.817a3.235 3.235 0 0 0-3.236 3.237 3.234 3.234 0 0 0 3.237 3.236 3.236 3.236 0 1 0 .004-6.473zm7.586 10.62c.647 1.3-.084 1.93-1.735 2.99-1.395.9-3.313 1.238-4.564 1.368l1.048 1.05 3.877 3.88c.59.59.59 1.543 0 2.133l-.177.18c-.59.59-1.544.59-2.134 0l-3.88-3.88-3.877 3.88c-.59.59-1.543.59-2.135 0l-.176-.18a1.505 1.505 0 0 1 0-2.132l3.88-3.877 1.042-1.046c-1.25-.127-3.19-.465-4.6-1.37-1.65-1.062-2.38-1.69-1.733-2.99.37-.747 1.4-1.367 2.768-.29C13.035 18.13 16 18.13 16 18.13s2.968 0 4.818-1.456c1.368-1.077 2.4-.457 2.768.29z"></path></svg></span>Odnoklassniki </a><a rel="nofollow noopener" class="a2a_i" href="/#oknotizie" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_oknotizie" style="background-color: rgb(136, 211, 45);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M13.333 24.975l-4.605-7.01H7.3v9.502h1.4V20.46l4.603 7.007h1.435v-9.5h-1.405m10.534 1.193c-.837-.896-1.927-1.352-3.24-1.352-1.315 0-2.41.455-3.25 1.352-.84.896-1.265 2.092-1.265 3.557 0 1.46.426 2.655 1.265 3.552.84.9 1.935 1.352 3.25 1.352 1.313 0 2.403-.455 3.24-1.353.835-.895 1.258-2.09 1.258-3.556s-.423-2.66-1.258-3.555zm-.248 3.557c0 1.14-.288 2.04-.85 2.67-.557.624-1.283.94-2.158.94-.866 0-1.59-.317-2.148-.945-.564-.632-.85-1.528-.85-2.665 0-1.142.287-2.04.852-2.67.56-.624 1.29-.94 2.17-.94.862 0 1.58.316 2.136.94.56.63.843 1.528.843 2.67zM11.39 4.375c-1.315 0-2.41.455-3.25 1.352-.84.895-1.265 2.09-1.265 3.556 0 1.46.426 2.656 1.265 3.554.84.9 1.934 1.354 3.25 1.354 1.313 0 2.403-.45 3.24-1.35.835-.893 1.258-2.09 1.258-3.554s-.423-2.66-1.258-3.556c-.837-.898-1.927-1.353-3.24-1.353zm2.992 4.908c0 1.142-.284 2.04-.846 2.67-.557.624-1.283.94-2.158.94-.865 0-1.59-.317-2.148-.944-.563-.636-.85-1.53-.85-2.67s.287-2.038.852-2.668c.562-.626 1.293-.943 2.172-.943.862 0 1.58.317 2.136.942.56.63.842 1.53.842 2.67zm4.14 2.015l.59-.59 2.11 3.326h1.705l-2.838-4.31 2.65-2.636H20.9l-2.377 2.5V4.534h-1.43v9.5h1.43" fill="#FFF"></path></svg></span>Oknotizie </a><a rel="nofollow noopener" class="a2a_i" href="/#outlook_com" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_outlook_com" style="background-color: rgb(0, 114, 198);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M19.52 8.29v5.5l1.92 1.208a.474.474 0 0 0 .212 0l8.27-5.574c0-.66-.613-1.134-.962-1.134h-9.44z"></path><path d="M19.52 15.84l1.755 1.204c.246.183.543 0 .543 0-.297.183 8.104-5.397 8.104-5.397V21.75c0 1.102-.704 1.562-1.496 1.562H19.52V15.84zm-9.075-2.535c-.6 0-1.073.282-1.426.842-.357.56-.53 1.305-.53 2.23 0 .936.173 1.677.53 2.22.345.546.81.82 1.38.82.59 0 1.053-.266 1.4-.795.342-.53.515-1.266.515-2.206 0-.984-.17-1.744-.502-2.288-.333-.55-.79-.823-1.37-.823z"></path><path d="M2.123 5.5v21.51l16.362 3.428V2.33L2.123 5.5zm10.95 14.387c-.693.91-1.594 1.367-2.706 1.367-1.082 0-1.967-.442-2.65-1.324-.68-.88-1.02-2.03-1.02-3.448 0-1.496.343-2.707 1.037-3.63.693-.926 1.614-1.388 2.754-1.388 1.08 0 1.955.438 2.62 1.324.667.885 1 2.05 1 3.495.004 1.496-.345 2.695-1.034 3.604z"></path></g></svg></span>Outlook.com </a><a rel="nofollow noopener" class="a2a_i" href="/#papaly" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_papaly" style="background-color: rgb(58, 192, 246);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.744 8.217c1.112 0 2.296.519 3.106 1.329l.185.185c1.361 1.361 1.402 4.432.042 5.792l-3.103 2.787L16 22.777l-4.974-4.467-3.103-2.787c-1.361-1.361-1.319-4.432.042-5.792l.185-.185c.81-.81 1.994-1.329 3.106-1.329.756 0 1.48.24 2.03.79L16 12.291l2.714-3.284c.55-.55 1.274-.79 2.03-.79m0-2.921c-1.58 0-3.035.585-4.096 1.646l-.098.098-.088.107-.462.558-.462-.559-.088-.106-.098-.098c-1.061-1.061-2.516-1.646-4.096-1.646-1.871 0-3.804.816-5.172 2.184l-.185.185c-2.515 2.515-2.535 7.43-.042 9.924l.055.055.058.052 3.103 2.787 4.974 4.467L16 26.704l1.952-1.753 4.974-4.467 3.103-2.787.058-.052.055-.055c2.494-2.494 2.473-7.409-.042-9.924l-.184-.186c-1.368-1.368-3.301-2.184-5.172-2.184z"></path></svg></span>Papaly </a><a rel="nofollow noopener" class="a2a_i" href="/#pinboard" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_pinboard" style="background-color: rgb(19, 65, 222);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M17.357 18.913l-5.01 5.014.88-4.5-6.588-8.075-3.48.044 4.314-4.313 4.035-4.04V6.85l7.795 6.403 4.502-.786-4.876 4.87 9.908 11.62"></path></svg></span>Pinboard </a><a rel="nofollow noopener" class="a2a_i" href="/#plurk" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_plurk" style="background-color: rgb(207, 104, 47);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.215 16.016h-8.43V9.7h8.43v6.316zm4.2 4.2V5.5H7.585v21h4.2v-6.285h12.63z"></path></svg></span>Plurk </a><a rel="nofollow noopener" class="a2a_i" href="/#pocket" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_pocket" style="background-color: rgb(238, 64, 86);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.005 6.244c2.927 0 5.854-.002 8.782 0 1.396.002 2.195.78 2.188 2.165-.015 2.483.116 4.985-.11 7.454-.75 8.204-10.027 12.607-16.91 8.064-3.086-2.037-4.82-4.926-4.917-8.673-.06-2.34-.034-4.684-.018-7.025.008-1.214.812-1.98 2.056-1.983 2.975-.01 5.952-.005 8.93-.007zm-5.037 5.483c-.867.093-1.365.396-1.62 1.025-.27.67-.078 1.256.417 1.732a529.74 529.74 0 0 0 5.09 4.838c.745.695 1.537.687 2.278-.01a473.74 473.74 0 0 0 4.93-4.686c.827-.797.91-1.714.252-2.38-.694-.704-1.583-.647-2.447.17-1.097 1.04-2.215 2.06-3.266 3.143-.485.492-.77.432-1.227-.027a87.392 87.392 0 0 0-3.39-3.225c-.325-.29-.77-.448-1.017-.584z"></path></svg></span>Pocket </a><a rel="nofollow noopener" class="a2a_i" href="/#polyvore" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_polyvore" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M7.82 5h9.84a6.68 6.68 0 0 1 4.72 1.65 6 6 0 0 1 1.79 4.65 6.76 6.76 0 0 1-1.6 4.49 6 6 0 0 1-4.9 1.91H10.8V27h-3zm11.56 3a6.4 6.4 0 0 0-2.69-.46H10.8v7.62h5.89a5.6 5.6 0 0 0 3.23-.85 3.4 3.4 0 0 0 1.24-3A3.31 3.31 0 0 0 19.38 8z" fill="#fff"></path></svg></span>Polyvore </a><a rel="nofollow noopener" class="a2a_i" href="/#print" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_print" style="background-color: rgb(1, 102, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M22.5 19.59V24h-13v-4.41z" fill="#fff" stroke="#fff" stroke-linecap="round" stroke-miterlimit="10" stroke-width="1.4"></path><path d="M9.44 21.12H6.38a.8.8 0 0 1-.87-.7v-7.35a.8.8 0 0 1 .88-.7h19.23a.8.8 0 0 1 .88.7v7.35a.8.8 0 0 1-.88.7h-3.06" fill="none" stroke="#fff" stroke-linecap="round" stroke-miterlimit="10" stroke-width="1.4"></path><path d="M22.5 12.32V8h-13v4.28" fill="#fff" stroke="#fff" stroke-linecap="round" stroke-miterlimit="10" stroke-width="1.4"></path></svg></span>Print </a><a rel="nofollow noopener" class="a2a_i" href="/#printfriendly" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_printfriendly" style="background-color: rgb(109, 159, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g stroke="#FFF" stroke-width="1.5" stroke-linecap="round" stroke-miterlimit="10"><path fill="none" d="M22.562 19.588v4.373H9.438v-4.37h13.124zm-13.124 1.53H6.375c-.483 0-.875-.313-.875-.7v-7.35c0-.387.392-.7.875-.7h19.25c.482 0 .875.313.875.7v7.35c0 .387-.393.7-.876.7h-3.062"></path><path fill="#FFF" d="M22.562 12.32V8.04H9.436v4.28"></path></g></svg></span>PrintFriendly </a><a rel="nofollow noopener" class="a2a_i" href="/#protopage_bookmarks" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_protopage" style="background-color: rgb(65, 63, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" stroke="#FFF" stroke-miterlimit="10" d="M17.866 14.47l7.626-1.048.574 3.078-7.68 1.038 3.54 7.058-2.804 1.418-3.614-7.23-5.873 5.557-2.144-2.29 5.74-5.42-6.86-3.6 1.595-2.697 6.808 3.595 1.3-7.375 3.1.546-1.303 7.373z"></path></svg></span>Protopage Bookmarks </a><a rel="nofollow noopener" class="a2a_i" href="/#pusha" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_pusha" style="background-color: rgb(0, 114, 184);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M29.27 22.188V8.068L17.207 14.92l3.838 2.33C15.715 24.144 5.898 29.306 0 31.964V32h19.635c3.682-4.865 7.03-11.46 7.03-11.46l2.604 1.648z"></path></svg></span>Pusha </a><a rel="nofollow noopener" class="a2a_i" href="/#qzone" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_qzone" style="background-color: rgb(43, 130, 217);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M27.996 12.83l-7.423-.737c-.566-.053-.694-.142-.87-.604l-3.175-7.045c-.29-.598-.765-.598-1.055 0l-3.384 7.04c-.23.393-.34.48-.898.534l-7.187.807c-.66.064-.808.493-.327.952l5.64 5.184c.266.25.27.355.195.697l-1.447 7.61c-.124.65.247.915.82.58l6.44-3.715c.45-.284.87-.293 1.31-.018l6.47 3.734c.575.334.948.07.826-.58L22.83 21.2c.66-.226 1.305-.5 1.69-.81l-.156.03c-2.29.547-5.438.872-8.355.872-1.08 0-2.128-.038-3.13-.11l-.006.005a39.39 39.39 0 0 1-2.53-.26c-.3-.05.026-.242.026-.242l7.76-5.513s.202-.126.002-.153c-3.188-.5-6.723-.627-10.042-.627h-.23c2.246-.51 5.07-.815 8.14-.815 1.81 0 3.54.105 5.11.296-.002.003.888.124 1.31.193.33.05.024.24.024.24l-7.77 5.384s-.18.107.015.136c2.39.337 5.332.457 7.98.49l-.118-.65c-.06-.38 0-.51.284-.78l5.478-5.12c.485-.455.34-.88-.318-.945z"></path></svg></span>Qzone </a><a rel="nofollow noopener" class="a2a_i" href="/#rediff" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_rediff" style="background-color: rgb(210, 0, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M22.506 6.004c-.336 0-.64 0-.92-.002L20.926 6c-1.742 0-2.418.07-3.738.923-.744.457-1.38 1.034-1.85 1.517V6.188a.182.182 0 0 0-.182-.184h-5.71c-.1 0-.183.082-.183.184v19.62c0 .115.115.23.232.18h5.71c.1 0 .18-.08.18-.18V14.933c0-2.584 1.85-2.916 3.464-2.916h3.7c.1 0 .183-.08.183-.182V6.188c-.05-.147-.172-.147-.287-.184h.056z"></path></svg></span>Rediff MyPage </a><a rel="nofollow noopener" class="a2a_i" href="/#refind" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_refind" style="background-color: rgb(20, 146, 239);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M16 21.256c-2.891 0-5.256-2.365-5.256-5.255 0-2.891 2.365-5.255 5.255-5.255 2.891 0 5.255 2.365 5.255 5.255.002 2.89-2.363 5.255-5.254 5.255z"></path><path d="M20.664 23.676A8.91 8.91 0 0 1 16 25c-4.95 0-9-4.05-9-9s4.05-9 9-9 9 4.05 9 9a8.912 8.912 0 0 1-1.302 4.628l2.293 1.991A11.908 11.908 0 0 0 28 16c0-6.6-5.4-12-12-12S4 9.4 4 16s5.4 12 12 12c2.456 0 4.745-.75 6.652-2.029l-1.988-2.295z"></path></g></svg></span>Refind </a><a rel="nofollow noopener" class="a2a_i" href="/#renren" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_renren" style="background-color: rgb(0, 94, 172);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M18.74 3.266a14.24 14.24 0 0 0-5.48 0c0 8.96-.07 16.176-9.26 21.662A14.406 14.406 0 0 0 8.322 29c3.595-2.168 5.688-4.736 7.69-8.275 2 3.54 4.07 6.107 7.69 8.275A14.07 14.07 0 0 0 28 24.928c-9.19-5.487-9.26-12.7-9.26-21.662z"></path></svg></span>Renren </a><a rel="nofollow noopener" class="a2a_i" href="/#sina_weibo" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_sina_weibo" style="background-color: rgb(230, 22, 45);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M13.92 24.99c-4.303.424-8.02-1.52-8.3-4.346-.278-2.827 2.987-5.463 7.292-5.888 4.304-.426 8.018 1.52 8.297 4.345.274 2.83-2.987 5.468-7.29 5.89m8.61-9.38c-.367-.11-.62-.184-.428-.663.416-1.046.458-1.946.01-2.59-.846-1.204-3.155-1.14-5.8-.03 0-.004-.834.362-.62-.297.406-1.31.345-2.407-.29-3.04-1.435-1.437-5.255.055-8.53 3.33C4.422 14.77 3 17.37 3 19.617c0 4.3 5.513 6.913 10.907 6.913 7.07 0 11.776-4.105 11.776-7.37 0-1.97-1.66-3.09-3.15-3.55m4.693-7.87a6.89 6.89 0 0 0-6.55-2.12h-.002a.997.997 0 0 0-.765 1.182.99.99 0 0 0 1.18.765 4.91 4.91 0 0 1 4.66 1.508 4.899 4.899 0 0 1 1.02 4.787.995.995 0 1 0 1.894.615v-.004a6.883 6.883 0 0 0-1.44-6.732m-2.622 2.37a3.343 3.343 0 0 0-3.192-1.03.852.852 0 0 0-.655 1.016.854.854 0 0 0 1.016.657v.003a1.655 1.655 0 0 1 1.564.502c.406.453.514 1.066.338 1.606h.005a.858.858 0 1 0 1.63.528 3.345 3.345 0 0 0-.7-3.28"></path><path d="M14.16 19.87c-.15.26-.484.383-.746.275-.256-.104-.335-.393-.19-.646.15-.255.47-.378.725-.276.26.094.35.386.21.644m-1.375 1.76c-.417.666-1.308.957-1.98.65-.66-.302-.855-1.072-.44-1.72.413-.645 1.274-.933 1.94-.653.673.287.888 1.054.48 1.724m1.564-4.7c-2.047-.533-4.364.49-5.254 2.293-.904 1.84-.028 3.884 2.04 4.552 2.144.69 4.67-.368 5.55-2.354.865-1.943-.216-3.943-2.336-4.49"></path></g></svg></span>Sina Weibo </a><a rel="nofollow noopener" class="a2a_i" href="/#sitejot" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_sitejot" style="background-color: rgb(255, 200, 8);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M23.506 8.08c.645 0 1.3.073 1.967.225.547.12.937.285 1.173.495.236.21.354.5.354.868 0 .232-.043.414-.13.547-.086.13-.272.29-.56.48-.333.224-.524.49-.573.794-.05.3-.094 1.48-.134 3.54-.018 1.99-.033 3.183-.048 3.58-.015.4-.05.786-.107 1.16-.167 1.1-.504 1.96-1.01 2.583-.414.51-.946.912-1.596 1.205-.65.293-1.335.44-2.054.44-.64 0-1.278-.11-1.92-.326a5.407 5.407 0 0 1-1.67-.902c-.39-.315-.7-.74-.93-1.278a4.326 4.326 0 0 1-.347-1.72c0-.75.186-1.37.548-1.863.362-.49.816-.738 1.363-.738.535 0 .99.207 1.363.62.375.42.56.938.56 1.555 0 .155-.033.42-.102.787-.01.075-.017.157-.017.247 0 .24.066.437.2.587.13.15.307.227.524.227.386 0 .696-.226.933-.677.233-.45.35-1.043.35-1.78l-.01-1.024-.016-2.76c-.052-1.676-.09-2.662-.116-2.96-.027-.296-.09-.538-.195-.725-.086-.15-.162-.247-.23-.29-.065-.047-.245-.118-.538-.216-.138-.038-.256-.144-.353-.315a1.114 1.114 0 0 1-.147-.555c0-.405.122-.73.366-.975.247-.242.646-.44 1.198-.59.62-.17 1.254-.25 1.91-.25zM10.202 8c.593 0 1.37.19 2.33.574a.785.785 0 0 0 .275.068c.058 0 .23-.068.518-.203.093-.05.19-.07.294-.07.38 0 .79.4 1.22 1.193.43.797.64 1.555.64 2.275 0 .42-.097.775-.29 1.065-.19.288-.426.434-.7.434-.23 0-.418-.06-.56-.18-.146-.12-.46-.48-.95-1.07-.69-.843-1.37-1.264-2.037-1.264-.322 0-.58.102-.772.305-.19.204-.29.47-.29.8 0 .654.44 1.147 1.32 1.477 1.193.46 1.955.793 2.288 1.003 1.466.934 2.2 2.305 2.2 4.114 0 1.6-.5 2.907-1.5 3.922C13.162 23.48 11.825 24 10.175 24a8.64 8.64 0 0 1-2.33-.344c-.816-.23-1.406-.497-1.77-.805-.274-.24-.523-.762-.745-1.57a8.972 8.972 0 0 1-.33-2.36c0-.368.06-.647.174-.834.143-.24.324-.36.543-.36.22 0 .43.146.63.438.12.165.405.695.855 1.59.215.42.553.768 1.016 1.048.463.28.94.418 1.437.418.426 0 .77-.104 1.035-.31a.964.964 0 0 0 .396-.794c0-.3-.103-.56-.31-.777-.208-.22-.544-.423-1.01-.61-.823-.338-1.45-.65-1.88-.93a5.844 5.844 0 0 1-1.21-1.064c-.908-1.053-1.362-2.246-1.362-3.583 0-.66.12-1.305.355-1.932s.57-1.17 1-1.627C7.63 8.53 8.81 8 10.2 8z" fill="#FFF"></path></svg></span>SiteJot </a><a rel="nofollow noopener" class="a2a_i" href="/#skype" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_skype" style="background-color: rgb(0, 175, 240);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M27.15 18c-.007.04-.012.084-.02.126l-.04-.24.06.113c.124-.678.19-1.37.19-2.06 0-1.53-.3-3.013-.892-4.41a11.273 11.273 0 0 0-2.43-3.602 11.288 11.288 0 0 0-8.012-3.32c-.72 0-1.443.068-2.146.203h-.005c.04.023.08.04.118.063l-.238-.037c.04-.01.08-.018.12-.026a6.717 6.717 0 0 0-3.146-.787 6.67 6.67 0 0 0-4.748 1.965A6.7 6.7 0 0 0 4 10.738c0 1.14.293 2.262.844 3.253.007-.04.012-.08.02-.12l.04.238-.06-.114c-.112.643-.17 1.3-.17 1.954a11.285 11.285 0 0 0 3.32 8.012c1.04 1.04 2.25 1.86 3.602 2.43 1.397.592 2.882.89 4.412.89.666 0 1.334-.06 1.985-.175-.038-.02-.077-.04-.116-.063l.242.04c-.046.01-.088.015-.13.02a6.68 6.68 0 0 0 3.3.87 6.661 6.661 0 0 0 4.743-1.963A6.666 6.666 0 0 0 28 21.26c0-1.145-.295-2.27-.85-3.264zm-11.098 4.885c-4.027 0-5.828-1.98-5.828-3.463 0-.76.562-1.294 1.336-1.294 1.723 0 1.277 2.474 4.49 2.474 1.647 0 2.556-.893 2.556-1.808 0-.55-.27-1.16-1.355-1.426l-3.58-.895c-2.88-.723-3.405-2.282-3.405-3.748 0-3.043 2.865-4.186 5.556-4.186 2.478 0 5.4 1.37 5.4 3.192 0 .783-.677 1.237-1.45 1.237-1.472 0-1.2-2.035-4.163-2.035-1.47 0-2.285.666-2.285 1.618 0 .95 1.16 1.254 2.17 1.484l2.65.587c2.905.647 3.64 2.342 3.64 3.94 0 2.47-1.895 4.318-5.726 4.318z"></path></svg></span>Skype </a><a rel="nofollow noopener" class="a2a_i" href="/#slashdot" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_slashdot" style="background-color: rgb(0, 66, 66);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M20.89 6h-5L8.61 26h5"></path><circle cx="20.89" cy="23.5" r="2.5"></circle></g></svg></span>Slashdot </a><a rel="nofollow noopener" class="a2a_i" href="/#sms" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_sms" style="background-color: rgb(108, 190, 69);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 3.543c-7.177 0-13 4.612-13 10.294 0 3.35 2.027 6.33 5.16 8.21 1.71 1.565 1.542 4.08-.827 6.41 2.874 0 7.445-1.698 8.462-4.34H16c7.176 0 13-4.605 13-10.285s-5.824-10.29-13-10.29zM9.045 17.376c-.73 0-1.45-.19-1.81-.388l.294-1.194c.384.2.98.398 1.6.398.66 0 1.01-.275 1.01-.692 0-.398-.302-.625-1.07-.9-1.06-.37-1.753-.957-1.753-1.886 0-1.09.91-1.924 2.415-1.924.72 0 1.25.152 1.63.322l-.322 1.166a3.037 3.037 0 0 0-1.336-.303c-.625 0-.93.284-.93.616 0 .41.36.59 1.186.9 1.127.42 1.658 1.01 1.658 1.91.003 1.07-.822 1.98-2.575 1.98zm9.053-.095l-.095-2.44a72.993 72.993 0 0 1-.057-2.626h-.028a35.41 35.41 0 0 1-.71 2.475l-.778 2.49h-1.128l-.682-2.473a29.602 29.602 0 0 1-.578-2.493h-.02c-.037.863-.065 1.85-.112 2.645l-.114 2.425H12.46l.407-6.386h1.924l.63 2.13c.2.74.397 1.536.54 2.285h.027a52.9 52.9 0 0 1 .607-2.293l.683-2.12h1.886l.35 6.386H18.1zm4.09.1c-.73 0-1.45-.19-1.81-.39l.293-1.194c.39.2.99.398 1.605.398.663 0 1.014-.275 1.014-.692 0-.396-.305-.623-1.07-.9-1.064-.37-1.755-.955-1.755-1.884 0-1.09.91-1.924 2.416-1.924.72 0 1.25.153 1.63.323l-.322 1.166a3.038 3.038 0 0 0-1.337-.303c-.625 0-.93.284-.93.616 0 .408.36.588 1.186.9 1.127.42 1.658 1.006 1.658 1.906.002 1.07-.823 1.98-2.576 1.98z"></path></svg></span>SMS </a><a rel="nofollow noopener" class="a2a_i" href="/#stocktwits" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_stocktwits" style="background-color: rgb(64, 87, 111);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M11.328 14.309l-1.793-.736c-1.303-.536-1.955-1.244-1.955-2.123 0-.637.249-1.166.746-1.587.498-.427 1.123-.642 1.878-.642.614 0 1.115.125 1.504.373.373.218.766.684 1.178 1.399l2.204-1.306c-1.166-2.024-2.788-3.037-4.863-3.037-1.539 0-2.828.459-3.866 1.376-1.039.909-1.559 2.039-1.559 3.391 0 2.005 1.232 3.528 3.698 4.569l1.738.722c.451.194.84.399 1.167.612.326.214.593.443.799.687.206.245.358.513.455.805.097.291.146.612.146.961 0 .871-.28 1.59-.84 2.156-.56.568-1.263.851-2.111.851-1.073 0-1.889-.389-2.449-1.166-.311-.405-.529-1.135-.653-2.193L4 20.028c.249 1.679.875 2.986 1.878 3.92 1.018.936 2.309 1.403 3.872 1.403 1.648 0 3.028-.544 4.139-1.634 1.102-1.082 1.653-2.451 1.653-4.109 0-1.237-.338-2.281-1.012-3.134-.676-.852-1.743-1.573-3.202-2.165zM28 7.023H17.037v2.571h4.14v15.425h2.695V9.594H28z" fill="#FFF"></path></svg></span>StockTwits </a><a rel="nofollow noopener" class="a2a_i" href="/#svejo" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_svejo" style="background-color: rgb(91, 212, 40);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M19.865 10.524c.31.16.48.29.48.29l.04-.02c.2-4.96-2.933-7.41-2.933-7.41-.688.87-1.147 2.21-1.387 3.04.26.11.51.24.76.38 2.13 1.19 2.83 2.92 3.04 3.72zm0 0c-.22-.8-.91-2.53-3.04-3.73-.25-.14-.5-.26-.76-.38-2.27-.99-4.73-.96-4.73-.96s0 2.92 3.08 4.95c.48-.17 1-.31 1.57-.42 1.81-.32 3.19.19 3.88.54zm-9.07 2.72c-.93 1.82 2.15 6.7 4.75 9.37 1.29 1.33 1.93 2.48 2.19 3.052 2.1.5 3.28-.29 3.42-.4l.07-.12c.93-1.94-1.46-4.818-3.19-7.068-2.14-2.78-2.7-4.832-2.7-4.832l-.01-.01c-3.5-1.582-4.53.008-4.53.008z"></path><path d="M21.176 25.274c-.15.11-1.32.9-3.42.4-1.1-.26-2.47-.88-4.07-2.15-2.31-1.818-4.03-3.43-5.2-5.53-.3-.528-.98-.568-1.37.2-.6 1.19-.67 4.5.7 6.44 0 0-.38 1.28.62 2.37.67.73 2.22 1.06 3.06.76l.17-.07s1.73 1.302 4.37.813c2.64-.49 4.33-1.73 5.18-3.24l-.04.007zm3.01-11.4s.38-1.28-.62-2.37c-.67-.73-2.22-1.06-3.06-.76l-.12.05-.04.02s-.17-.13-.48-.28c-.69-.36-2.07-.87-3.89-.53-.57.102-1.09.25-1.57.42-1.68.59-2.83 1.542-3.51 2.66l-.1.17s1.02-1.59 4.53-.01c.84.382 1.83.94 2.978 1.75 2.41 1.692 4.03 3.432 5.2 5.53.3.53.98.57 1.37-.198.612-1.2.682-4.512-.69-6.45z"></path></g></svg></span>Svejo </a><a rel="nofollow noopener" class="a2a_i" href="/#symbaloo_feeds" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_symbaloo" style="background-color: rgb(109, 168, 247);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M7 7h4.5v4.5H7zm6.75 0h4.5v4.5h-4.5zm6.75 0H25v4.5h-4.5zM7 13.75h4.5v4.5H7zm6.75 0h4.5v4.5h-4.5zm6.75 0H25v4.5h-4.5zM7 20.5h4.5V25H7zm6.75 0h4.5V25h-4.5zm6.75 0H25V25h-4.5z" fill="#FFF"></path></svg></span>Symbaloo Feeds </a><a rel="nofollow noopener" class="a2a_i" href="/#telegram" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_telegram" style="background-color: rgb(44, 165, 224);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M25.515 6.896L6.027 14.41c-1.33.534-1.322 1.276-.243 1.606l5 1.56 1.72 5.66c.226.625.115.873.77.873.506 0 .73-.235 1.012-.51l2.43-2.363 5.056 3.734c.93.514 1.602.25 1.834-.863l3.32-15.638c.338-1.363-.52-1.98-1.41-1.577z"></path></svg></span>Telegram </a><a rel="nofollow noopener" class="a2a_i" href="/#threema" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_threema" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M11.4 25.941a1.734 1.734 0 1 1-3.467 0 1.734 1.734 0 0 1 3.467 0m12.667 0a1.734 1.734 0 1 1-3.467 0 1.734 1.734 0 0 1 3.467 0m-6.334 0a1.734 1.734 0 1 1-3.467 0 1.734 1.734 0 0 1 3.467 0M16.082 8.716h-.262c-.886 0-1.507.682-1.507 1.568v2.015h3.372v-2.015c.001-.886-.718-1.568-1.603-1.568z"></path><path d="M16 4.325c-6.075 0-11 3.752-11 8.381 0 1.924.851 3.697 2.283 5.111.49.484.607 1.231.283 1.839l-1.479 2.776 4.39-1.384a6.183 6.183 0 0 1 1.863-.284c.383 0 .766.034 1.145.102.807.144 1.649.22 2.514.22 6.075 0 11-3.752 11-8.381S22.075 4.325 16 4.325zm3.793 12.432c0 .368-.409.6-.777.6H12.85c-.368 0-.643-.231-.643-.6v-3.733c0-.368.275-.725.643-.725h.2v-2.015c0-1.585 1.187-2.832 2.771-2.832h.262c1.583 0 2.868 1.247 2.868 2.832v2.015h.066c.368 0 .777.357.777.725v3.733z"></path></g></svg></span>Threema </a><a rel="nofollow noopener" class="a2a_i" href="/#trello" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_trello" style="background-color: rgb(0, 121, 191);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M26.5 4h-21A1.5 1.5 0 0 0 4 5.5v21A1.5 1.5 0 0 0 5.5 28h21a1.5 1.5 0 0 0 1.5-1.5v-21A1.5 1.5 0 0 0 26.5 4zM14.44 22.12c0 .825-.675 1.5-1.5 1.5H8.62c-.825 0-1.5-.675-1.5-1.5V8.62c0-.825.675-1.5 1.5-1.5h4.32c.825 0 1.5.675 1.5 1.5v13.5zm10.44-6c0 .825-.675 1.5-1.5 1.5h-4.32c-.825 0-1.5-.675-1.5-1.5v-7.5c0-.825.675-1.5 1.5-1.5h4.32c.825 0 1.5.675 1.5 1.5v7.5z"></path></svg></span>Trello </a><a rel="nofollow noopener" class="a2a_i" href="/#tuenti" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_tuenti" style="background-color: rgb(0, 117, 201);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M13.907 19.803l-2.463 5.062a1.836 1.836 0 0 1-2.463.847 1.844 1.844 0 0 1-.844-2.465l2.46-5.062a1.844 1.844 0 0 1 3.314 1.618zm-.053-7.497a2.394 2.394 0 0 1-1.682.698 2.383 2.383 0 0 1-2.378-2.378c0-.626.255-1.236.7-1.68.885-.888 2.48-.888 3.36 0 .442.444.696 1.054.696 1.68 0 .626-.254 1.235-.696 1.68zm9.834 7.17a16.055 16.055 0 0 1-2.445 5.73 1.81 1.81 0 0 1-2.522.478 1.815 1.815 0 0 1-.473-2.52 12.532 12.532 0 0 0 1.896-4.448c.38-1.775.38-3.654 0-5.432a12.506 12.506 0 0 0-1.895-4.445 1.82 1.82 0 0 1 .48-2.527 1.815 1.815 0 0 1 2.522.48c1.177 1.727 2 3.655 2.442 5.73a16.68 16.68 0 0 1-.002 6.95z" fill="#FFF"></path></svg></span>Tuenti </a><a rel="nofollow noopener" class="a2a_i" href="/#twiddla" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_twiddla" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M10.5 5.688l1.375 1.375-4.812 4.812L5.688 10.5A3.438 3.438 0 0 1 10.5 5.688zm2.406 2.406l-4.812 4.812 12.72 12.72L27 27l-1.375-6.188-12.72-12.718zm-.095 3.533l9.63 9.625-1.187 1.183-9.624-9.625 1.185-1.183z"></path></svg></span>Twiddla </a><a rel="nofollow noopener" class="a2a_i" href="/#typepad_post" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_typepad" style="background-color: rgb(210, 222, 97);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 8.875c-6.627 0-12 3.225-12 7.202 0 .844.342 2.21.787 2.407.447.196 1.67.683 12.523-3.836 0 0-9.096 4.09-9.83 5.85-.253.605 2.154 2.627 8.52 2.627 6.626 0 12-3.148 12-7.125s-5.374-7.125-12-7.125z"></path></svg></span>TypePad Post </a><a rel="nofollow noopener" class="a2a_i" href="/#viadeo" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_viadeo" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M15.107 4s2.838 1.88 3.326 7.05c0 0 .957 12.423-5.47 14.858 0 0 .577.106 1.224.076 0 0 7.947-5.012 4.3-14.742 0 0-1.09-3.396-3.38-7.242zm4.15 8.483s-1.48-2.29.397-4.18c0 0 .658-.725 2.537-1.202 0 0 1.88-.4 2.897-2.553 0 0 2.105 2.94.81 6.807 0 0-.64 1.742-1.976 2.538-1.01.6-2.535.788-4.133-.776 0 0 4.83-2.644 5.01-4.884 0 0-1.483 3.334-5.54 4.256z"></path><path d="M19.256 12.483s-1.48-2.29.398-4.18c0 0 .658-.725 2.537-1.202 0 0 1.88-.4 2.897-2.553 0 0 2.105 2.94.81 6.807 0 0-.64 1.742-1.976 2.538-1.01.6-2.535.788-4.133-.776 0 0 4.83-2.644 5.01-4.884 0 0-1.483 3.334-5.54 4.256zm1.23 6.95c0 1.19-.283 2.3-.85 3.33-.57 1.03-1.34 1.825-2.306 2.384-.967.56-2.03.84-3.186.84-1.156 0-2.22-.28-3.186-.84-.97-.56-1.736-1.354-2.305-2.383a6.787 6.787 0 0 1-.853-3.33c0-1.847.625-3.42 1.87-4.723 1.247-1.3 2.74-1.95 4.474-1.95.824 0 1.596.15 2.313.45.072-.754.336-1.456.63-2.03a8.578 8.578 0 0 0-2.936-.49c-2.504 0-4.58.92-6.22 2.77-1.495 1.675-2.24 3.65-2.24 5.933 0 2.3.79 4.31 2.366 6.03C9.63 27.14 11.666 28 14.15 28c2.48 0 4.508-.86 6.086-2.58 1.578-1.72 2.367-3.73 2.367-6.03 0-1.233-.22-2.374-.65-3.427a6.21 6.21 0 0 1-1.982.797c.347.816.52 1.707.52 2.674z"></path></g></svg></span>Viadeo </a><a rel="nofollow noopener" class="a2a_i" href="/#viber" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_viber" style="background-color: rgb(124, 82, 158);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M22.57 27.22a7.39 7.39 0 0 1-1.14-.32 29 29 0 0 1-16-16.12c-1-2.55 0-4.7 2.66-5.58a2 2 0 0 1 1.39 0c1.12.41 3.94 4.3 4 5.46a2 2 0 0 1-1.16 1.78 2 2 0 0 0-.66 2.84A10.3 10.3 0 0 0 17 20.55a1.67 1.67 0 0 0 2.35-.55c1.07-1.62 2.38-1.54 3.82-.54.72.51 1.45 1 2.14 1.55.93.75 2.1 1.37 1.55 2.94a5.21 5.21 0 0 1-4.29 3.27zM17.06 4.79A10.42 10.42 0 0 1 26.79 15c0 .51.18 1.27-.58 1.25s-.54-.78-.6-1.29c-.7-5.52-3.23-8.13-8.71-9-.45-.07-1.15 0-1.11-.57.05-.87.87-.54 1.27-.6z" fill="#fff" fill-rule="evenodd"></path><path d="M24.09 14.06c-.05.38.17 1-.45 1.13-.83.13-.67-.64-.75-1.13-.56-3.36-1.74-4.59-5.12-5.35-.5-.11-1.27 0-1.15-.8s.82-.48 1.35-.42a6.9 6.9 0 0 1 6.12 6.57z" fill="#fff" fill-rule="evenodd"></path><path d="M21.52 13.45c0 .43 0 .87-.53.93s-.6-.26-.64-.64a2.47 2.47 0 0 0-2.26-2.43c-.42-.07-.82-.2-.63-.76.13-.38.47-.41.83-.42a3.66 3.66 0 0 1 3.23 3.32z" fill="#fff" fill-rule="evenodd"></path></svg></span>Viber </a><a rel="nofollow noopener" class="a2a_i" href="/#vk" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_vk" style="background-color: rgb(88, 126, 163);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill-rule="evenodd" clip-rule="evenodd" fill="#FFF" d="M15.764 22.223h1.315s.394-.044.6-.262c.184-.2.18-.574.18-.574s-.03-1.764.79-2.023c.81-.255 1.844 1.705 2.942 2.46.832.57 1.464.445 1.464.445l2.936-.04s1.538-.097.81-1.304c-.06-.1-.426-.894-2.186-2.526-1.843-1.71-1.594-1.434.624-4.39 1.353-1.804 1.893-2.902 1.724-3.374-.16-.45-1.153-.33-1.153-.33l-3.306.02s-.247-.034-.428.074c-.178.108-.293.356-.293.356s-.522 1.394-1.223 2.58c-1.47 2.5-2.06 2.633-2.3 2.476-.563-.36-.42-1.454-.42-2.23 0-2.423.365-3.435-.72-3.696-.357-.085-.623-.143-1.544-.15-1.182-.014-2.18.003-2.743.28-.378.185-.667.595-.49.62.218.027.713.13.975.49.34.46.33 1.496.33 1.496s.193 2.852-.46 3.206c-.442.245-1.056-.252-2.37-2.52-.67-1.163-1.18-2.446-1.18-2.446s-.1-.24-.273-.37c-.212-.155-.506-.204-.506-.204l-3.145.02s-.473.015-.647.22c-.154.183-.01.56-.01.56s2.46 5.757 5.245 8.657c2.553 2.66 5.454 2.485 5.454 2.485z"></path></svg></span>VK </a><a rel="nofollow noopener" class="a2a_i" href="/#wanelo" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_wanelo" style="background-color: rgb(156, 176, 146);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#fff" d="M4 4h7.5v7.5H4zm16.5 0H28v7.5h-7.5zm-8.25 0h7.5v7.5h-7.5zM4 20.5h7.5V28H4zm8.25-8.25h7.5v7.5h-7.5zm0 8.25h7.5V28h-7.5zM4 12.25h7.5v7.5H4zm16.5 0H28v7.5h-7.5z"></path></svg></span>Wanelo </a><a rel="nofollow noopener" class="a2a_i" href="/#wechat" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_wechat" style="background-color: rgb(123, 179, 46);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M20.674 12.458c-2.228.116-4.165.792-5.738 2.318-1.59 1.542-2.315 3.43-2.116 5.772-.87-.108-1.664-.227-2.462-.294-.276-.023-.602.01-.836.14-.774.438-1.517.932-2.397 1.482.16-.73.266-1.37.45-1.985.137-.45.074-.7-.342-.994-2.673-1.89-3.803-4.714-2.958-7.624.78-2.69 2.697-4.323 5.302-5.173 3.555-1.16 7.55.022 9.712 2.845a6.632 6.632 0 0 1 1.38 3.516zm-10.253-.906c.025-.532-.44-1.01-.984-1.027a.997.997 0 0 0-1.038.964.984.984 0 0 0 .977 1.02 1.017 1.017 0 0 0 1.05-.96zm5.35-1.028c-.55.01-1.01.478-1 1.012.01.554.466.987 1.03.98a.982.982 0 0 0 .99-1.01.992.992 0 0 0-1.02-.982z"></path><path d="M25.68 26.347c-.705-.314-1.352-.785-2.042-.857-.686-.072-1.408.324-2.126.398-2.187.224-4.147-.386-5.762-1.88-3.073-2.842-2.634-7.2.92-9.53 3.16-2.07 7.795-1.38 10.022 1.493 1.944 2.51 1.716 5.837-.658 7.94-.687.61-.934 1.11-.493 1.917.086.148.095.336.14.523zm-8.03-7.775c.448 0 .818-.35.835-.795a.835.835 0 0 0-.83-.865.845.845 0 0 0-.84.86c.016.442.388.8.834.8zm5.176-1.658a.83.83 0 0 0-.824.794c-.02.47.347.858.813.86.45 0 .807-.34.823-.79a.825.825 0 0 0-.812-.864z"></path></g></svg></span>WeChat </a><a rel="nofollow noopener" class="a2a_i" href="/#wykop" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_wykop" style="background-color: rgb(54, 125, 169);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M23.54 3.5H8.46A4.96 4.96 0 0 0 3.5 8.46v15.08a4.96 4.96 0 0 0 4.96 4.96h15.08a4.96 4.96 0 0 0 4.96-4.96V8.46a4.96 4.96 0 0 0-4.96-4.96zM10.93 25.98L5 13.927l4.018-1.977 4.94 10.044 2.01-.988-4.94-10.043 4.016-1.977 4.942 10.043 2.01-.99L17.05 7.994l4.02-1.977L27 18.07l-16.07 7.91z"></path></svg></span>Wykop </a><a rel="nofollow noopener" class="a2a_i" href="/#xing" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_xing" style="background-color: rgb(22, 91, 102);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M7.647 8.74c-.208 0-.384.072-.473.216-.09.147-.075.338.02.53l2.342 4.053c.003.004.003.01 0 .02l-3.68 6.49c-.095.19-.09.38 0 .53.09.14.245.23.454.23h3.46c.52 0 .768-.347.945-.667l3.736-6.607-2.38-4.15c-.17-.308-.43-.65-.96-.65H7.65zM22.187 4c-.516 0-.74.325-.926.66l-7.7 13.657 4.92 9.023c.17.307.437.66.967.66h3.458c.208 0 .37-.08.46-.222.092-.148.09-.344-.008-.535l-4.88-8.915c-.005-.007-.005-.017 0-.023l7.665-13.55c.096-.19.097-.387.006-.535-.088-.142-.25-.22-.46-.22H22.19z" fill="#FFF"></path></svg></span>XING </a><a rel="nofollow noopener" class="a2a_i" href="/#yahoo_bookmarks" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yahoo" style="background-color: rgb(64, 0, 144);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M21.495 6.097c1.09.31 2.213.28 3.304 0l-7.42 12.09v9.91a4.366 4.366 0 0 0-1.37-.22c-.47 0-.937.065-1.404.22v-9.91L7.19 6.097c1.09.28 2.213.31 3.304 0l5.516 8.788 5.483-8.787z"></path></svg></span>Yahoo Bookmarks </a><a rel="nofollow noopener" class="a2a_i" href="/#yahoo_mail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yahoo" style="background-color: rgb(64, 0, 144);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M21.495 6.097c1.09.31 2.213.28 3.304 0l-7.42 12.09v9.91a4.366 4.366 0 0 0-1.37-.22c-.47 0-.937.065-1.404.22v-9.91L7.19 6.097c1.09.28 2.213.31 3.304 0l5.516 8.788 5.483-8.787z"></path></svg></span>Yahoo Mail </a><a rel="nofollow noopener" class="a2a_i" href="/#yahoo_messenger" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yim" style="background-color: rgb(64, 0, 144);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 27c6.074 0 11-4.926 11-11 0-6.075-4.926-11-11-11S5 9.925 5 16c0 6.074 4.926 11 11 11zm2.808-17.016a1.432 1.432 0 1 1 .003 2.865 1.432 1.432 0 0 1-.003-2.865zm-5.443 0a1.433 1.433 0 1 1 0 2.865 1.433 1.433 0 0 1 0-2.865zm-5.1 4.583h15.47v2.866h-.032a7.726 7.726 0 1 1-15.402 0h-.03v-2.866zm14.42 2.56H9.13v-1.764h13.554v1.764z"></path></svg></span>Yahoo Messenger </a><a rel="nofollow noopener" class="a2a_i" href="/#yoolink" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yoolink" style="background-color: rgb(162, 197, 56);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16 4C9.37 4 4 9.373 4 16c0 6.628 5.372 12 12 12 6.626 0 12-5.372 12-12 0-6.627-5.374-12-12-12zm5.81 7.1c-.128.275-.346.655-.655 1.14l-3.694 5.942v4.908c0 .64-.14 1.118-.42 1.436a1.38 1.38 0 0 1-1.08.474c-.44 0-.8-.157-1.075-.47-.274-.313-.412-.794-.412-1.44v-4.91l-3.603-5.855c-.316-.528-.54-.94-.67-1.235-.13-.295-.197-.553-.197-.77 0-.36.133-.67.398-.93s.59-.39.98-.39c.41 0 .72.12.93.364.213.243.53.727.953 1.45l2.758 4.697 2.79-4.693c.17-.287.31-.53.423-.727.114-.198.24-.384.378-.556.14-.172.29-.305.46-.396.166-.09.37-.136.613-.136.376 0 .688.13.94.386.252.258.378.554.378.892 0 .275-.064.55-.19.823z"></path></svg></span>Yoolink </a><a rel="nofollow noopener" class="a2a_i" href="/#yummly" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yummly" style="background-color: rgb(225, 97, 32);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M27.127 21.682c-.015-.137-.132-.213-.216-.236-.21-.06-.43-.01-1.06-.29-.51-.23-2.875-1.37-6.13-1.746l2.357-13.426c.105-.602.1-1.087-.09-1.394-.277-.45-.886-.573-1.586-.514-.545.05-.98.25-1.07.312a.325.325 0 0 0-.145.288c.023.253.22.45.057 1.425-.032.21-.802 4.505-1.453 8.14-1.724 1.038-4.018 1.527-4.488.905-.228-.31-.177-.89.04-1.757.05-.193 1.06-4.03 1.347-5.135.54-2.105.13-4.035-2.05-4.23-1.88-.17-3.676.935-4.216 1.51-.39.415-.26.916.09 1.52.275.473.642.78.735.836.115.07.263.07.32.01.63-.71 1.775-1.243 2.173-.915.35.29.216.83.08 1.35 0 0-1.227 4.606-1.723 6.526-.366 1.417-.007 2.705 1.027 3.32.77.473 1.914.435 2.816.32 1.96-.24 3.11-1.066 3.296-1.208l-.363 2.02s-2.214.2-4.027 1.286c-2.383 1.428-3.345 4.673-1.82 6.347 1.526 1.674 4.193 1.04 5.277.308 1.045-.7 2.407-2.18 3.023-5.535 3.596.184 4.53 2.046 6.063 2.113 1.1.048 1.876-1.014 1.737-2.142zm-12.23 3.214c-.51.297-1.03.337-1.35-.03-.337-.388-.435-2.5 2.914-3.13.005 0-.523 2.56-1.56 3.16z"></path></svg></span>Yummly </a><i class="a2a_i"></i><i class="a2a_i"></i></div><div class="a2a_full_footer"><a href="https://www.addtoany.com" title="Share Buttons" rel="noopener" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_a2a" style="background-color:#0166ff"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M14 7h4v18h-4z"></path><path d="M7 14h18v4H7z"></path></g></svg></span>AddToAny</a></div></div><div id="a2apage_dropdown" class="a2a_menu a2a_mini" onmouseenter="a2a.miniEnterStay()" onmouseleave="a2a.miniLeaveDelay()" tabindex="-1" aria-label="Share" style="display:none"><div id="a2apage_title_container" class="a2a_menu_title_container" style="display:none"><div id="a2apage_title" class="a2a_menu_title"></div></div><div class="a2a_mini_services" id="a2apage_mini_services"><a rel="nofollow noopener" class="a2a_i" href="/#facebook" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_facebook" style="background-color: rgb(59, 89, 152);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M17.78 27.5V17.008h3.522l.527-4.09h-4.05v-2.61c0-1.182.33-1.99 2.023-1.99h2.166V4.66c-.375-.05-1.66-.16-3.155-.16-3.123 0-5.26 1.905-5.26 5.405v3.016h-3.53v4.09h3.53V27.5h4.223z"></path></svg></span>Facebook </a><a rel="nofollow noopener" class="a2a_i" href="/#twitter" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_twitter" style="background-color: rgb(85, 172, 238);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M28 8.557a9.913 9.913 0 0 1-2.828.775 4.93 4.93 0 0 0 2.166-2.725 9.738 9.738 0 0 1-3.13 1.194 4.92 4.92 0 0 0-3.593-1.55 4.924 4.924 0 0 0-4.794 6.049c-4.09-.21-7.72-2.17-10.15-5.15a4.942 4.942 0 0 0-.665 2.477c0 1.71.87 3.214 2.19 4.1a4.968 4.968 0 0 1-2.23-.616v.06c0 2.39 1.7 4.38 3.952 4.83-.414.115-.85.174-1.297.174-.318 0-.626-.03-.928-.086a4.935 4.935 0 0 0 4.6 3.42 9.893 9.893 0 0 1-6.114 2.107c-.398 0-.79-.023-1.175-.068a13.953 13.953 0 0 0 7.55 2.213c9.056 0 14.01-7.507 14.01-14.013 0-.213-.005-.426-.015-.637.96-.695 1.795-1.56 2.455-2.55z"></path></svg></span>Twitter </a><a rel="nofollow noopener" class="a2a_i" href="/#google_plus" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_google_plus" style="background-color: rgb(221, 75, 57);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M27 15h-2v-2h-2v2h-2v2h2v2h2v-2h2m-15-2v2.4h3.97c-.16 1.03-1.2 3.02-3.97 3.02-2.39 0-4.34-1.98-4.34-4.42s1.95-4.42 4.34-4.42c1.36 0 2.27.58 2.79 1.08l1.9-1.83C15.47 9.69 13.89 9 12 9c-3.87 0-7 3.13-7 7s3.13 7 7 7c4.04 0 6.72-2.84 6.72-6.84 0-.46-.05-.81-.11-1.16H12z" fill="#FFF"></path></svg></span>Google+ </a><a rel="nofollow noopener" class="a2a_i" href="/#pinterest" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_pinterest" style="background-color: rgb(189, 8, 28);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M16.539 4.5c-6.277 0-9.442 4.5-9.442 8.253 0 2.272.86 4.293 2.705 5.046.303.125.574.005.662-.33.061-.231.205-.816.27-1.06.088-.331.053-.447-.191-.736-.532-.627-.873-1.439-.873-2.591 0-3.338 2.498-6.327 6.505-6.327 3.548 0 5.497 2.168 5.497 5.062 0 3.81-1.686 7.025-4.188 7.025-1.382 0-2.416-1.142-2.085-2.545.397-1.674 1.166-3.48 1.166-4.689 0-1.081-.581-1.983-1.782-1.983-1.413 0-2.548 1.462-2.548 3.419 0 1.247.421 2.091.421 2.091l-1.699 7.199c-.505 2.137-.076 4.755-.039 5.019.021.158.223.196.314.077.13-.17 1.813-2.247 2.384-4.324.162-.587.929-3.631.929-3.631.46.876 1.801 1.646 3.227 1.646 4.247 0 7.128-3.871 7.128-9.053.003-3.918-3.317-7.568-8.361-7.568z"></path></svg></span>Pinterest </a><a rel="nofollow noopener" class="a2a_i" href="/#email" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_email" style="background-color: rgb(1, 102, 255);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M26 21.25v-9s-9.1 6.35-9.984 6.68C15.144 18.616 6 12.25 6 12.25v9c0 1.25.266 1.5 1.5 1.5h17c1.266 0 1.5-.22 1.5-1.5zm-.015-10.765c0-.91-.265-1.235-1.485-1.235h-17c-1.255 0-1.5.39-1.5 1.3l.015.14s9.035 6.22 10 6.56c1.02-.395 9.985-6.7 9.985-6.7l-.015-.065z"></path></svg></span>Email </a><a rel="nofollow noopener" class="a2a_i" href="/#linkedin" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_linkedin" style="background-color: rgb(0, 123, 181);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M6.227 12.61h4.19v13.48h-4.19V12.61zm2.095-6.7a2.43 2.43 0 0 1 0 4.86c-1.344 0-2.428-1.09-2.428-2.43s1.084-2.43 2.428-2.43m4.72 6.7h4.02v1.84h.058c.56-1.058 1.927-2.176 3.965-2.176 4.238 0 5.02 2.792 5.02 6.42v7.395h-4.183v-6.56c0-1.564-.03-3.574-2.178-3.574-2.18 0-2.514 1.7-2.514 3.46v6.668h-4.187V12.61z" fill="#FFF"></path></svg></span>LinkedIn </a><a rel="nofollow noopener" class="a2a_i" href="/#reddit" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_reddit" style="background-color: rgb(255, 69, 0);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M28.543 15.774a2.953 2.953 0 0 0-2.951-2.949 2.882 2.882 0 0 0-1.9.713 14.075 14.075 0 0 0-6.85-2.044l1.38-4.349 3.768.884a2.452 2.452 0 1 0 .24-1.176l-4.274-1a.6.6 0 0 0-.709.4l-1.659 5.224a14.314 14.314 0 0 0-7.316 2.029 2.908 2.908 0 0 0-1.872-.681 2.942 2.942 0 0 0-1.618 5.4 5.109 5.109 0 0 0-.062.765c0 4.158 5.037 7.541 11.229 7.541s11.22-3.383 11.22-7.541a5.2 5.2 0 0 0-.053-.706 2.963 2.963 0 0 0 1.427-2.51zm-18.008 1.88a1.753 1.753 0 0 1 1.73-1.74 1.73 1.73 0 0 1 1.709 1.74 1.709 1.709 0 0 1-1.709 1.711 1.733 1.733 0 0 1-1.73-1.711zm9.565 4.968a5.573 5.573 0 0 1-4.081 1.272h-.032a5.576 5.576 0 0 1-4.087-1.272.6.6 0 0 1 .844-.854 4.5 4.5 0 0 0 3.238.927h.032a4.5 4.5 0 0 0 3.237-.927.6.6 0 1 1 .844.854zm-.331-3.256a1.726 1.726 0 1 1 1.709-1.712 1.717 1.717 0 0 1-1.712 1.712z" fill="#fff"></path></svg></span>Reddit </a><a rel="nofollow noopener" class="a2a_i" href="/#tumblr" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_tumblr" style="background-color: rgb(53, 70, 92);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M20.775 21.962c-.37.177-1.08.33-1.61.345-1.598.043-1.907-1.122-1.92-1.968v-6.217h4.007V11.1H17.26V6.02h-2.925s-.132.044-.144.15c-.17 1.556-.895 4.287-3.923 5.378v2.578h2.02v6.522c0 2.232 1.647 5.404 5.994 5.33 1.467-.025 3.096-.64 3.456-1.17l-.96-2.846z"></path></svg></span>Tumblr </a></div><div id="a2apage_cols_container" class="a2a_cols_container"><div class="a2a_col1" id="a2apage_col1"></div><div id="a2apage_2_col1" style="display:none"><a rel="nofollow noopener" class="a2a_i" href="/#google_gmail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_gmail" style="background-color: rgb(221, 83, 71);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#fff" opacity=".25" d="M4.958 6.569h22.338V25.43H4.958z"></path><path d="M28.709 7.321a1.7 1.7 0 0 0-1.409-.752h-.077l-1.1.8-.082.06-9.952 7.271L5.961 7.3l-1-.733H4.7A1.7 1.7 0 0 0 3 8.273v15.454a1.676 1.676 0 0 0 .069.481A1.7 1.7 0 0 0 4.7 25.431h1.261V11.36l7.35 5.368 2.416 1.764.445.326 2.778-2.029 7.088-5.177v13.819H27.3a1.7 1.7 0 0 0 1.634-1.223 1.675 1.675 0 0 0 .066-.481V8.273a1.7 1.7 0 0 0-.291-.952z" fill="#fff"></path></svg></span>Google Gmail </a><a rel="nofollow noopener" class="a2a_i" href="/#aol_mail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_aol" style="background-color: rgb(42, 42, 42);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path d="M17.334 13.26c-2.315 0-4.067 1.8-4.067 4.027 0 2.35 1.824 4.03 4.067 4.03 2.243 0 4.062-1.68 4.062-4.03 0-2.228-1.744-4.027-4.062-4.027zm0 2.127c1-.007 1.82.847 1.82 1.9 0 1.048-.82 1.9-1.82 1.9s-1.818-.853-1.818-1.9c0-1.053.817-1.9 1.818-1.9zm11.59 4.518c0 .778-.63 1.412-1.41 1.412a1.41 1.41 0 1 1 1.41-1.412zm-4.104 1.418h-2.216v-10.28h2.216v10.28zM9.33 11.04s2.585 6.79 3.862 10.13c.015.037.028.078.047.132-.06.006-.106.01-.15.01-.83.002-1.665-.003-2.498.004-.12.002-.17-.04-.204-.156-.116-.385-.247-.766-.365-1.147-.032-.11-.074-.153-.193-.153-1.066.006-2.132.006-3.2 0-.1 0-.142.03-.173.13-.127.405-.26.81-.39 1.21-.02.076-.05.117-.136.117-.873-.006-1.75-.004-2.623-.004-.016 0-.036-.005-.07-.012.023-.06.04-.115.064-.17 1.287-3.306 3.91-10.085 3.91-10.085h2.12zm-.023 6.674c-.343-1.147-.68-2.274-1.02-3.4h-.03l-1.017 3.4h2.067z" fill="#FFF"></path></svg></span>AOL Mail </a><a rel="nofollow noopener" class="a2a_i" href="/#outlook_com" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_outlook_com" style="background-color: rgb(0, 114, 198);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M19.52 8.29v5.5l1.92 1.208a.474.474 0 0 0 .212 0l8.27-5.574c0-.66-.613-1.134-.962-1.134h-9.44z"></path><path d="M19.52 15.84l1.755 1.204c.246.183.543 0 .543 0-.297.183 8.104-5.397 8.104-5.397V21.75c0 1.102-.704 1.562-1.496 1.562H19.52V15.84zm-9.075-2.535c-.6 0-1.073.282-1.426.842-.357.56-.53 1.305-.53 2.23 0 .936.173 1.677.53 2.22.345.546.81.82 1.38.82.59 0 1.053-.266 1.4-.795.342-.53.515-1.266.515-2.206 0-.984-.17-1.744-.502-2.288-.333-.55-.79-.823-1.37-.823z"></path><path d="M2.123 5.5v21.51l16.362 3.428V2.33L2.123 5.5zm10.95 14.387c-.693.91-1.594 1.367-2.706 1.367-1.082 0-1.967-.442-2.65-1.324-.68-.88-1.02-2.03-1.02-3.448 0-1.496.343-2.707 1.037-3.63.693-.926 1.614-1.388 2.754-1.388 1.08 0 1.955.438 2.62 1.324.667.885 1 2.05 1 3.495.004 1.496-.345 2.695-1.034 3.604z"></path></g></svg></span>Outlook.com </a><a rel="nofollow noopener" class="a2a_i" href="/#yahoo_mail" target="_blank"><span class="a2a_svg a2a_s__default a2a_s_yahoo" style="background-color: rgb(64, 0, 144);"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M21.495 6.097c1.09.31 2.213.28 3.304 0l-7.42 12.09v9.91a4.366 4.366 0 0 0-1.37-.22c-.47 0-.937.065-1.404.22v-9.91L7.19 6.097c1.09.28 2.213.31 3.304 0l5.516 8.788 5.483-8.787z"></path></svg></span>Yahoo Mail </a><a rel="nofollow noopener" class="a2a_i a2a_emailer a2a_email_client" href="/#email_app" target="" id="a2apage_email_client"><span class="a2a_svg a2a_s__default a2a_s_email"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><path fill="#FFF" d="M26 21.25v-9s-9.1 6.35-9.984 6.68C15.144 18.616 6 12.25 6 12.25v9c0 1.25.266 1.5 1.5 1.5h17c1.266 0 1.5-.22 1.5-1.5zm-.015-10.765c0-.91-.265-1.235-1.485-1.235h-17c-1.255 0-1.5.39-1.5 1.3l.015.14s9.035 6.22 10 6.56c1.02-.395 9.985-6.7 9.985-6.7l-.015-.065z"></path></svg></span>Email app </a></div><div class="a2a_clear"></div></div><div class="a2apage_wide a2a_wide"><a href="" id="a2apage_show_more_less" class="a2a_menu_show_more_less a2a_more" title="Show all"><span class="a2a_svg a2a_s__default a2a_s_a2a" style="background-color:#0166ff"><svg focusable="false" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 32 32"><g fill="#FFF"><path d="M14 7h4v18h-4z"></path><path d="M7 14h18v4H7z"></path></g></svg></span>More…</a></div></div><div style="height: 1px; width: 1px; position: absolute; z-index: 100000; top: 0px; visibility: hidden;"><iframe id="a2apage_sm_ifr" height="1" width="1" title="AddToAny Utility Frame" transparency="true" allowtransparency="true" frameborder="0" src="https://static.addtoany.com/menu/sm.17.html#type=page&amp;event=load&amp;url=https%3A%2F%2Fwww.lewrockwell.com%2F2018%2F01%2Fdr-david-brownstein%2Fzika-schmika-iv-where-is-our-billion-dollars%2F&amp;referrer=" style="height: 1px; width: 1px; border: 0px; left: 0px; top: 0px; position: absolute; z-index: 100000; display: none;" kwframeid="4"></iframe></div></div>
		<div class="footer inner">
			<div class="hr clearfix"><div class="menu-main-menu-container"><ul id="menu-main-menu-1" class="nav"><li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441343"><a href="/archives/">Archives</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441344"><a href="/columnists">Authors</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441520"><a href="/lrc-blog/">Blog</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-has-children menu-item-441345"><a href="#">Books &amp; Resources</a>
<ul class="sub-menu">
	<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-655521"><a href="https://donate.lewrockwell.com/collections/books-by-lew-rockwell">Lew Rockwell Books</a></li>
	<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441351"><a href="https://www.lewrockwell.com/books-resources/ron-paul-books/">Ron Paul Books</a></li>
	<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441350"><a href="https://www.lewrockwell.com/books-resources/murray-n-rothbard-library-and-resources/">Murray N. Rothbard Library &amp; Resources</a></li>
	<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-441352"><a href="https://www.mises.org/">Mises Institute</a></li>
	<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-441349"><a href="https://www.lewrockwell.com/books-resources/lrc-best-selling-books/">LRC Bestselling Books</a></li>
</ul>
</li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-548797"><a href="/political-theatre/">Political Theatre</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-547557"><a href="/podcast/">Podcasts</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-523610"><a href="/store/">Store</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-47847"><a href="https://www.lewrockwell.com/about/">About</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-521748"><a rel="nofollow" href="/contact/">Contact</a></li>
<li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-521749"><a rel="nofollow" href="https://donate.lewrockwell.com/products/donations">Donate</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-523960"><a href="https://www.lewrockwell.com/advertise-on-lrc/">Advertise</a></li>
<li class="menu-item menu-item-search"><form role="search" method="get" class="search-form" action="https://www.lewrockwell.com/"><label><span class="screen-reader-text">Search for:</span><input type="search" class="search-field" placeholder="Search …" value="" name="s" title="Search for:"></label><input type="submit" class="search-submit" value="Search"></form></li></ul></div></div>
			<div class="clearfix f-bottom">
				<div class="f-copyright"><a target="_blank" rel="nofollow" href="https://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="creativecommons.org" src="https://www.lewrockwell.com//wp-content/themes/lrc/images/creative-commons-4.0.png"></a></div>
				<!-- footer_menu_2 Cache --><div class="menu-footer-mini-menu-container"><ul id="menu-footer-mini-menu" class="f-links"><li id="menu-item-468390" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-468390"><a href="https://www.lewrockwell.com/privacy/">Privacy Policy</a></li>
</ul></div>			</div>
		</div>
		<script>
    jQuery(document).ready(function () {
		jQuery.post('https://www.lewrockwell.com?ga_action=googleanalytics_get_script', {action: 'googleanalytics_get_script'}, function(response) {
			var F = new Function ( response );
			return( F() );
		});
    });
</script><script type="text/javascript">
/* <![CDATA[ */
var ThemeFrontEnd = {"isprint":"false","ismobile":"false"};
/* ]]> */
</script>
<script type="text/javascript" src="https://www.lewrockwell.com/wp-content/themes/lrc/js/lrc-frontend.js?ver=20171102193351" async="async"></script>
<script type="text/javascript" src="https://www.lewrockwell.com/wp-includes/js/wp-embed.js?ver=4.9.4"></script>
<script type="text/javascript" src="https://www.lewrockwell.com/wp-includes/js/comment-reply.js?ver=4.9.4"></script>
		<script>(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)})(window,document,'script','https://www.google-analytics.com/analytics.js','ga');ga('create', 'UA-372872-1', 'auto');ga('send', 'pageview');</script>
	
<iframe scrolling="no" frameborder="0" allowtransparency="true" src="https://platform.twitter.com/widgets/widget_iframe.36c0c29c73929bf937f4c70adb1a29e4.html?origin=https%3A%2F%2Fwww.lewrockwell.com" style="display: none;" kwframeid="5"></iframe><iframe id="rufous-sandbox" scrolling="no" frameborder="0" allowtransparency="true" allowfullscreen="true" style="position: absolute; visibility: hidden; display: none; width: 0px; height: 0px; padding: 0px; border: none;" title="Twitter analytics iframe" kwframeid="7"></iframe><iframe name="oauth2relay1774561472" id="oauth2relay1774561472" src="https://accounts.google.com/o/oauth2/postmessageRelay?parent=https%3A%2F%2Fwww.lewrockwell.com&amp;jsh=m%3B%2F_%2Fscs%2Fapps-static%2F_%2Fjs%2Fk%3Doz.gapi.en.WfSuwKgCRTA.O%2Fm%3D__features__%2Fam%3DAQE%2Frt%3Dj%2Fd%3D1%2Frs%3DAGLTcCO0KuWtpejjm06jRxdbUYOYGDplMQ#rpctoken=515464906&amp;forcesecure=1" tabindex="-1" aria-hidden="true" kwframeid="8" style="width: 1px; height: 1px; position: absolute; top: -100px;"></iframe></body>

In [4]:
from pprint import pprint

with open('C:\\Users\\mkennedy\\firstpage3.json') as data_file:    
    data = json.load(data_file)
pprint(data)

JSONDecodeError: Expecting value: line 13 column 1 (char 156)

In [5]:
import scrapy
from scrapy.crawler import CrawlerProcess

# Make a list for the urls to be stored into from the dataframe
url_list = []
for entry in df_data['URL']:
    url_list.append(entry)
    
print(url_list)

class WikiSpider(scrapy.Spider):
    name = "WS"
    
    # Here is where we insert our API call.
    start_urls = url_list

    # Identifying the information we want from the query response and extracting it using xpath.
    def parse(self, response):
        for item in response.xpath('//lh'):
            # The ns code identifies the type of page the link comes from.  '0' means it is a Wikipedia entry.
            # Other codes indicate links from 'Talk' pages, etc.  Since we are only interested in entries, we filter:
            if item.xpath('@ns').extract_first() == '0':
                yield {
                    'text': item.xpath('//*[@id="post-669657"]/div[3]').extract_first() 
                    ,'body': item.xpath('//*[@id="single-post"]').extract_first()
                    }
        # Getting the information needed to continue to the next ten entries.
        next_page = response.xpath('continue/@lhcontinue').extract_first()
        
        # Recursively calling the spider to process the next ten entries, if they exist.
        if next_page is not None:
            next_page = '{}&lhcontinue={}'.format(self.start_urls[0],next_page)
            yield scrapy.Request(next_page, callback=self.parse)
            
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'PythonLinks.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # We use CLOSESPIDER_PAGECOUNT to limit our scraper to the first 100 links.    
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(WikiSpider)
process.start()
print('First 100 links extracted!')

['https://www.lewrockwell.com/2018/01/dr-david-brownstein/zika-schmika-iv-where-is-our-billion-dollars/', 'https://www.sciencedaily.com/releases/2017/12/171213161105.htm', 'https://www.eurekalert.org/pub_releases/2017-12/uow-mib121317.php', 'https://www.reuters.com/article/us-health-zika-paralysis/u-s-study-sheds-light-on-how-zika-causes-nerve-disorder-idUSKBN1E735E', 'https://www.fiercepharma.com/vaccines/takeda-s-zika-vaccine-gets-fda-fast-track-though-virus-no-longer-emergency', 'https://www.laboratoryequipment.com/news/2017/12/cdc-report-infants-affected-zika-not-reaching-developmental-milestones', 'https://in.reuters.com/article/us-health-zika-paralysis/u-s-study-sheds-light-on-how-zika-causes-nerve-disorder-idINKBN1E735E', 'https://medicalxpress.com/news/2017-11-zika-outbreak.html', 'http://outbreaknewstoday.com/los-angeles-1st-sexually-transmitted-zika-infection-reported-13985/', 'http://outbreaknewstoday.com/birth-defects-seen-parts-u-s-local-zika-spread-cdc-83806/', 'https://u

ReactorNotRestartable: 